# AI OCR Benchmark System

This notebook demonstrates a comprehensive OCR (Optical Character Recognition) system that can extract structured data from images using multiple AI models including Google Gemini, OpenAI GPT-4V, and Claude.

## Features
- ✅ **Model Agnostic** - Switch between different AI models
- ✅ **Batch Processing** - Process entire directories of images
- ✅ **Custom Prompts** - Flexible prompt engineering for different document types
- ✅ **JSON Output** - Structured data extraction with automatic parsing
- ✅ **Image Optimization** - Automatic image resizing and compression
- ✅ **Scoring System** - Benchmark and compare model performance

## Library Imports and Dependencies

This section imports all necessary libraries for the OCR system including LangChain components, image processing libraries, and AI model clients.

In [1]:
import os
import base64
from pathlib import Path
from typing import List, Dict, Optional, Union
from dotenv import load_dotenv
import json
import re
from datetime import datetime
from openai import OpenAI
from pydantic import SecretStr

# LangChain imports
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

# For image processing
from PIL import Image
import io

print("All required libraries imported successfully!")

All required libraries imported successfully!


## OCR Benchmark Class Definition

This section defines the main OCRBenchmark class with global configuration variables, utility functions for JSON parsing, and the complete implementation of the model-agnostic OCR system.

In [2]:
# Global Configuration Variables for Image Processing
# Set to False to disable image resizing/optimization
ENABLE_IMAGE_OPTIMIZATION = True
# Resize ratio (0.5 = 50% of original size, 1.0 = no resize)
IMAGE_RESIZE_RATIO = 0.5
MAX_IMAGE_DIMENSION = 1024  # Maximum width or height in pixels
MAX_FILE_SIZE_MB = 4  # Maximum file size in MB before optimization
JPEG_QUALITY = 70  # JPEG compression quality (1-100)

# JSON Parsing and Formatting Utilities
def parse_json_from_text(text: str):
    """
    Extract and parse JSON from text that might contain markdown code blocks or other formatting.
    """
    try:
        # First try to parse directly
        return json.loads(text)
    except json.JSONDecodeError:
        pass
    
    # Try to extract JSON from markdown code blocks
    json_patterns = [
        r'```json\s*(\{.*?\})\s*```',  # ```json ... ```
        r'```\s*(\{.*?\})\s*```',      # ``` ... ```
        r'(\{.*?\})',                   # Just the JSON object
    ]
    
    for pattern in json_patterns:
        matches = re.findall(pattern, text, re.DOTALL)
        for match in matches:
            try:
                return json.loads(match.strip())
            except json.JSONDecodeError:
                continue
    
    # If all parsing fails, return the original text
    return text

def format_extraction_result(result: Dict) -> Dict:
    """
    Format the extraction result with parsed JSON data for better readability.
    """
    if result.get("success") and "extracted_data" in result:
        try:
            # Try to parse the extracted data as JSON
            parsed_data = parse_json_from_text(result["extracted_data"])
            
            # If successfully parsed, add it as a separate field
            if isinstance(parsed_data, (dict, list)):
                result["extracted_data_parsed"] = parsed_data
                result["extraction_format"] = "json"
            else:
                result["extraction_format"] = "text"
                
        except Exception as e:
            result["extraction_format"] = "text"
            result["parsing_error"] = str(e)
    
    return result


class OCRBenchmark:
    """
    A model-agnostic OCR system using LangChain that can extract data from images
    using different AI models (Gemini, OpenAI GPT-4V, Claude).
    """

    def __init__(self, model_name: str = "gemini", temperature: float = 0.1):
        """
        Initialize the OCR Benchmark system.

        Args:
            model_name: The model to use ("gemini", "openai", "anthropic")
            temperature: Temperature for model generation
        """
        # Load environment variables
        load_dotenv()

        self.model_name = model_name
        self.temperature = temperature
        self.model = None

        # Supported image formats
        self.supported_formats = {'.png', '.jpg', '.jpeg', '.webp', '.gif'}

        # MIME type mapping for correct image format handling
        self.mime_type_map = {
            '.jpg': 'jpeg',
            '.jpeg': 'jpeg',
            '.png': 'png',
            '.webp': 'webp',
            '.gif': 'gif'
        }

        # Image optimization settings (now using global variables)
        self.enable_optimization = ENABLE_IMAGE_OPTIMIZATION
        self.resize_ratio = IMAGE_RESIZE_RATIO
        self.max_image_size = (MAX_IMAGE_DIMENSION, MAX_IMAGE_DIMENSION)
        self.max_file_size_mb = MAX_FILE_SIZE_MB
        self.jpeg_quality = JPEG_QUALITY

        # Try to initialize model, but don't fail if API key is missing
        try:
            self.model = self._initialize_model()
        except ValueError as e:
            print(f"⚠️ Warning: {e}")
            print(
                f"Model '{model_name}' will be initialized when needed if API key becomes available.")

    def _initialize_model(self):
        """Initialize the specified model."""
        if self.model_name.lower() == "gemini":
            api_key = os.getenv("GOOGLE_API_KEY")
            if not api_key or api_key == "your_google_api_key_here":
                raise ValueError(
                    "GOOGLE_API_KEY not found or not set. Please set your GOOGLE_API_KEY in the .env file")
            return ChatGoogleGenerativeAI(
                model="gemini-2.0-flash",  # Updated to newer model
                temperature=self.temperature,
                google_api_key=api_key,
                timeout=60  # Set reasonable timeout
            )

        elif self.model_name.lower() == "openai":
            api_key = os.getenv("OPENAI_API_KEY")
            if not api_key or api_key == "your_openai_api_key_here":
                raise ValueError(
                    "OPENAI_API_KEY not found or not set. Please set your OPENAI_API_KEY in the .env file")
            return ChatOpenAI(
                model="gpt-4o",
                temperature=self.temperature,
                timeout=60
            )

        elif self.model_name.lower() == "anthropic":
            return ChatAnthropic(
                model_name="claude-3-5-sonnet-20241022",
                temperature=self.temperature,
                timeout=60,
                stop=None
            )
            
        elif self.model_name.lower() == "huggingface":
            api_key = os.getenv("HF_TOKEN")
            if not api_key:
                raise ValueError(
                    "HF_TOKEN not found. Please set your Hugging Face API token in the .env file"
                )
            return ChatOpenAI(
                model="Qwen/Qwen2.5-VL-72B-Instruct:nebius",  # or make this configurable
                temperature=self.temperature,
                base_url="https://router.huggingface.co/v1",
                api_key=SecretStr(api_key),
                timeout=60,
    )
            
        else:
            raise ValueError(f"Unsupported model: {self.model_name}")

    def _optimize_image(self, image_path: str) -> bytes:
        """Optimize image size and quality for API processing."""
        try:
            # Check if optimization is enabled
            if not self.enable_optimization:
                print(f"🔧 Image optimization disabled - using original file")
                with open(image_path, "rb") as f:
                    return f.read()

            # Check file size first
            file_size_mb = os.path.getsize(image_path) / (1024 * 1024)
            print(f"📏 Original file size: {file_size_mb:.2f} MB")

            with Image.open(image_path) as img:
                # Convert to RGB if necessary
                if img.mode in ('RGBA', 'P'):
                    img = img.convert('RGB')

                original_size = img.size
                print(
                    f"📐 Original dimensions: {original_size[0]}x{original_size[1]}")

                # Apply more aggressive optimization for HuggingFace due to strict size limits
                if self.model_name.lower() == "huggingface":
                    # Use smaller resize ratio for HuggingFace
                    hf_resize_ratio = min(self.resize_ratio, 0.5)  # Max 50% of original size
                    hf_max_dimension = min(self.max_image_size[0], 900)  # Max 900px
                    hf_quality = min(self.jpeg_quality, 50)  # Lower quality
                    
                    new_width = int(img.size[0] * hf_resize_ratio)
                    new_height = int(img.size[1] * hf_resize_ratio)
                    img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
                    print(f"🔧 HuggingFace aggressive resize by ratio {hf_resize_ratio}: {img.size[0]}x{img.size[1]}")
                    
                    # # Further resize if still too large
                    # if img.size[0] > hf_max_dimension or img.size[1] > hf_max_dimension:
                    #     img.thumbnail((hf_max_dimension, hf_max_dimension), Image.Resampling.LANCZOS)
                    #     print(f"🔧 HuggingFace thumbnail resize to: {img.size[0]}x{img.size[1]}")
                    
                    # Save with lower quality for HuggingFace
                    img_byte_arr = io.BytesIO()
                    if Path(image_path).suffix.lower() in ['.jpg', '.jpeg']:
                        img.save(img_byte_arr, format='JPEG', quality=hf_quality, optimize=True)
                        print(f"💾 HuggingFace JPEG quality: {hf_quality}")
                    else:
                        img.save(img_byte_arr, format='PNG', optimize=True)
                        print(f"💾 HuggingFace PNG optimized")
                    
                    optimized_data = img_byte_arr.getvalue()
                    optimized_size_mb = len(optimized_data) / (1024 * 1024)
                    print(f"✅ HuggingFace optimized size: {optimized_size_mb:.2f} MB")
                    
                    # # If still too large, try even more aggressive compression
                    # if optimized_size_mb > 0.3:  # 300KB limit for HuggingFace
                    #     print(f"⚠️ Still too large, applying ultra-compression...")
                    #     # Ultra-aggressive resizing
                    #     ultra_dimension = 512
                    #     img.thumbnail((ultra_dimension, ultra_dimension), Image.Resampling.LANCZOS)
                    #     print(f"🔧 Ultra resize to: {img.size[0]}x{img.size[1]}")
                        
                    #     # Ultra-low quality
                    #     img_byte_arr = io.BytesIO()
                    #     if Path(image_path).suffix.lower() in ['.jpg', '.jpeg']:
                    #         img.save(img_byte_arr, format='JPEG', quality=60, optimize=True)
                    #         print(f"💾 Ultra JPEG quality: 60")
                    #     else:
                    #         img.save(img_byte_arr, format='PNG', optimize=True)
                        
                    #     optimized_data = img_byte_arr.getvalue()
                    #     optimized_size_mb = len(optimized_data) / (1024 * 1024)
                    #     print(f"✅ Ultra-compressed size: {optimized_size_mb:.2f} MB")
                    
                    return optimized_data

                # Standard optimization for other models
                # Apply resize ratio if specified
                if self.resize_ratio != 1.0:
                    new_width = int(img.size[0] * self.resize_ratio)
                    new_height = int(img.size[1] * self.resize_ratio)
                    img = img.resize((new_width, new_height),
                                     Image.Resampling.LANCZOS)
                    print(
                        f"🔧 Resized by ratio {self.resize_ratio}: {img.size[0]}x{img.size[1]}")

                # Additional resize if still too large
                elif (img.size[0] > self.max_image_size[0] or
                      img.size[1] > self.max_image_size[1] or
                      file_size_mb > self.max_file_size_mb):

                    img.thumbnail(self.max_image_size,
                                  Image.Resampling.LANCZOS)
                    print(
                        f"🔧 Thumbnail resize to: {img.size[0]}x{img.size[1]}")

                # Save to bytes with optimized quality
                img_byte_arr = io.BytesIO()

                # Choose format and quality based on original
                if Path(image_path).suffix.lower() in ['.jpg', '.jpeg']:
                    img.save(img_byte_arr, format='JPEG',
                             quality=self.jpeg_quality, optimize=True)
                    print(f"💾 JPEG quality: {self.jpeg_quality}")
                else:
                    img.save(img_byte_arr, format='PNG', optimize=True)
                    print(f"💾 PNG optimized")

                optimized_data = img_byte_arr.getvalue()
                optimized_size_mb = len(optimized_data) / (1024 * 1024)
                print(f"✅ Final optimized size: {optimized_size_mb:.2f} MB")

                return optimized_data

        except Exception as e:
            print(f"⚠️ Image optimization failed, using original: {e}")
            # Fallback to original file
            with open(image_path, "rb") as f:
                return f.read()

    def _ensure_model_initialized(self):
        """Ensure model is initialized before use."""
        if self.model is None:
            try:
                self.model = self._initialize_model()
            except ValueError as e:
                raise ValueError(
                    f"Cannot initialize model '{self.model_name}': {e}")

    def encode_image(self, image_path: str) -> str:
        """Encode image to base64 string."""
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    def extract_data_from_image(self, image_path: str, prompt: str) -> Dict:
        """
        Extract data from a single image using the specified prompt.

        Args:
            image_path: Path to the image file
            prompt: Custom prompt for data extraction

        Returns:
            Dictionary containing the extracted data and metadata
        """
        try:
            print(f"🚀 Starting image processing...")

            # Ensure model is initialized
            self._ensure_model_initialized()

            # Validate image file
            if not os.path.exists(image_path):
                return {
                    "error": f"Image file not found: {image_path}",
                    "success": False
                }

            # Check file extension
            file_ext = Path(image_path).suffix.lower()
            if file_ext not in self.supported_formats:
                return {
                    "error": f"Unsupported image format: {file_ext}",
                    "success": False
                }

            # Get correct MIME type
            mime_type = self.mime_type_map.get(file_ext, file_ext[1:])
            print(
                f"🔍 File extension: {file_ext} → MIME type: image/{mime_type}")

            # Optimize image
            print(f"🖼️ Optimizing image...")
            image_data = self._optimize_image(image_path)

            # Encode to base64
            print(f"📝 Encoding to base64...")
            base64_image = base64.b64encode(image_data).decode()
            base64_size_mb = len(base64_image) / (1024 * 1024)
            print(f"📊 Base64 size: {base64_size_mb:.2f} MB")

            # # Check if base64 is still too large for HuggingFace
            # if self.model_name.lower() == "huggingface" and base64_size_mb > 0.5:
            #     print(f"⚠️ Base64 still too large for HuggingFace ({base64_size_mb:.2f} MB > 0.5 MB)")
            #     return {
            #         "error": f"Image too large for HuggingFace API even after optimization. Base64 size: {base64_size_mb:.2f} MB. Try using a smaller or lower quality image.",
            #         "success": False,
            #         "image_path": image_path,
            #         "base64_size_mb": base64_size_mb
            #     }

            # Construct data URL
            data_url = f"data:image/{mime_type};base64,{base64_image}"

            # Prepare message
            print(f"🤖 Sending to {self.model_name.upper()}...")
            message = HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": data_url}
                    }
                ]
            )

            # Get response with timeout handling
            if self.model is not None:
                import time
                start_time = time.time()
                response = self.model.invoke([message])
                end_time = time.time()
                print(f"⏱️ API call completed in {end_time - start_time:.2f} seconds")
            else:
                raise ValueError("Model not initialized")

            result = {
                "image_path": image_path,
                "model_used": self.model_name,
                "prompt": prompt,
                "extracted_data": response.content,
                "success": True,
                "timestamp": datetime.now().isoformat(),
                "processing_time_seconds": end_time - start_time if 'end_time' in locals() else None
            }
            
            # Format the result with parsed JSON data
            return format_extraction_result(result)

        except Exception as e:
            print(f"❌ Error during processing: {str(e)}")
            return {
                "image_path": image_path,
                "error": str(e),
                "success": False,
                "timestamp": datetime.now().isoformat()
            }

    def get_images_from_directory(self, directory_path: str) -> List[str]:
        """
        Get all supported image files from a directory.

        Args:
            directory_path: Path to the directory containing images

        Returns:
            List of image file paths
        """
        directory = Path(directory_path)
        if not directory.exists():
            raise ValueError(f"Directory not found: {directory_path}")

        image_files = []
        for file_path in directory.iterdir():
            if file_path.is_file() and file_path.suffix.lower() in self.supported_formats:
                image_files.append(str(file_path))

        return sorted(image_files)

    def save_results(self, results: List[Dict], output_file: str):
        """Save results to a JSON file."""
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=2, ensure_ascii=False)

    def save_results_formatted(self, results: List[Dict], output_file: str, 
                             include_parsed_data: bool = True):
        """
        Save results with enhanced formatting and optional separation of parsed data.
        
        Args:
            results: List of OCR results
            output_file: Path to save the formatted results
            include_parsed_data: Whether to include parsed JSON data separately
        """
        # Create formatted results
        formatted_results = []
        
        for result in results:
            formatted_result = result.copy()
            
            # If we have parsed JSON data, optionally restructure
            if include_parsed_data and "extracted_data_parsed" in result:
                # Move parsed data to top level for easier access
                formatted_result["structured_data"] = result["extracted_data_parsed"]
                
                # Keep original text for reference but make it shorter in display
                original_text = result["extracted_data"]
                if len(original_text) > 200:
                    formatted_result["extracted_data_preview"] = original_text[:200] + "..."
                    formatted_result["extracted_data_full"] = original_text
                    del formatted_result["extracted_data"]
                
            formatted_results.append(formatted_result)
        
        # Save with proper formatting
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(formatted_results, f, indent=2, ensure_ascii=False)
        
        print(f"📄 Formatted results saved to: {output_file}")
        
        # Also create a summary file with just the structured data
        if include_parsed_data:
            summary_file = output_file.replace('.json', '_structured_only.json')
            structured_only = []
            
            for result in formatted_results:
                if "structured_data" in result:
                    structured_only.append({
                        "image_path": result["image_path"],
                        "model_used": result["model_used"],
                        "success": result["success"],
                        "structured_data": result["structured_data"],
                        "timestamp": result["timestamp"]
                    })
            
            with open(summary_file, 'w', encoding='utf-8') as f:
                json.dump(structured_only, f, indent=2, ensure_ascii=False)
            
            print(f"📋 Structured data summary saved to: {summary_file}")

    def process_directory(self, directory_path: str, prompt: str,
                          output_file: Optional[str] = None,
                          save_formatted: bool = True) -> List[Dict]:
        """
        Process all images in a directory with enhanced result formatting.
        """
        image_files = self.get_images_from_directory(directory_path)

        if not image_files:
            print(f"No supported image files found in {directory_path}")
            return []

        print(f"Found {len(image_files)} image(s) to process...")

        results = []
        for i, image_path in enumerate(image_files, 1):
            print(f"Processing image {i}/{len(image_files)}: {Path(image_path).name}")
            result = self.extract_data_from_image(image_path, prompt)
            results.append(result)

            if result["success"]:
                print(f"✅ Successfully processed {Path(image_path).name}")
                # Show preview of structured data if available
                if "extracted_data_parsed" in result:
                    print(f"📊 Extracted structured data format: {result.get('extraction_format', 'unknown')}")
            else:
                print(f"❌ Failed to process {Path(image_path).name}: {result.get('error', 'Unknown error')}")

        # Save results with enhanced formatting
        if output_file:
            if save_formatted:
                self.save_results_formatted(results, output_file, include_parsed_data=True)
            else:
                self.save_results(results, output_file)
            print(f"Results saved to {output_file}")

        return results

    def change_model(self, model_name: str):
        """Change the model being used."""
        self.model_name = model_name
        self.model = None  # Reset model
        try:
            self.model = self._initialize_model()
            print(f"✅ Model changed to: {model_name}")
        except ValueError as e:
            print(f"⚠️ Warning: Failed to initialize {model_name}: {e}")
            print(f"Model will be initialized when needed if API key becomes available.")

    def check_api_keys(self) -> Dict[str, bool]:
        """Check which API keys are available."""
        keys_status = {}

        google_key = os.getenv("GOOGLE_API_KEY")
        keys_status["gemini"] = bool(
            google_key and google_key != "your_google_api_key_here")

        openai_key = os.getenv("OPENAI_API_KEY")
        keys_status["openai"] = bool(
            openai_key and openai_key != "your_openai_api_key_here")

        anthropic_key = os.getenv("ANTHROPIC_API_KEY")
        keys_status["anthropic"] = bool(
            anthropic_key and anthropic_key != "your_anthropic_api_key_here")

        return keys_status


print("OCRBenchmark class defined successfully!")
print(
    f"🔧 Global settings: Optimization={ENABLE_IMAGE_OPTIMIZATION}, Resize ratio={IMAGE_RESIZE_RATIO}")

OCRBenchmark class defined successfully!
🔧 Global settings: Optimization=True, Resize ratio=0.5


## Sample Prompts Creation

This section creates various pre-defined prompts for different OCR tasks including payslip extraction, cash expense extraction, general OCR, and structured data extraction.

In [ ]:
# Example usage and demonstration
def create_sample_prompts():
    """Create sample prompts for different OCR tasks."""
    return {
        "general_ocr": "Extract all text from this image and format it nicely. Preserve the structure and layout as much as possible.",

        "payslip_extraction": """
        This is a payslip/salary slip image. Please extract the following information in JSON format:
        {
          "employee_name": "",
          "employee_id": "",
          "pay_period": "",
          "gross_salary": "",
          "net_salary": "",
          "deductions": [],
          "company_name": ""
        }
        If any field is not found, use null.
        """,

        "structured_data": """
        Extract all structured data from this image and organize it in a clear, readable format. 
        Identify tables, forms, or any structured information and present it clearly.
        """,

        "key_value_pairs": """
        Extract all key-value pairs from this image. Present them as:
        Key: Value
        Key: Value
        etc.
        """,

        "cash_expense_extraction": """
You are an expert document processing assistant. Your task is to extract structured data from invoice/receipt images and return the information in a specific JSON format.

## Instructions:
1. Analyze the provided image carefully to identify all relevant financial information
2. Extract the data according to the JSON schema provided below
3. If any required field is not visible or unclear in the image, use reasonable defaults or null values
4. For monetary amounts, use numbers without currency symbols
5. For dates, use the format found in the document or convert to YYYY-MM-DD if possible
6. Be precise with calculations - ensure totals match the extracted item details
7. The document may contain text in both English and Bangla (Bengali). For Bangla text, provide English transliteration (not translation) - write the Bangla words using English letters to show how they sound. For Bangla numbers convert them to
    English numerals (e.g., "১২৩" becomes "123").
## Required JSON Schema:
Return your response as a valid JSON object following this exact structure:

{
  "payment_details": {
    "supplier": "string - Name of the supplier/vendor",
    "payment_account": "string - Account used for payment",
    "payment_date": "string - Date of payment",
    "payment_method": "string - Mode of payment (cash, card, bank transfer, etc.)",
    "ref_no": "string - Reference/invoice number",
    "tags": ["array of relevant tags"]
  },
  "item_details": [
    {
      "category": "string - Expense category",
      "description": "string - Item description (optional)",
      "quantity": "number - Quantity of items",
      "unit_price": "number - Price per unit",
      "total": "number - Total amount for this item"
    },
    {
      "category": "string - Expense category for item 2",
      "description": "string - Item description (optional)",
      "quantity": "number - Quantity of items",
      "unit_price": "number - Price per unit", 
      "total": "number - Total amount for this item"
    }
    // ... additional items as found in the document
  ],
  "attachment": "string - File reference if mentioned",
  "memo": "string - Any additional notes or memo",
  "totals": {
    "sub_total": "number - Subtotal before tax and discount",
    "sales_tax": {
      "is_percentage": "boolean - true if tax is %, false if fixed amount",
      "tax": "number - Tax rate / VAT rate (%)",
      "amount": "number - Calculated tax amount"
    },
    "discount": {
      "is_percentage": "boolean - true if discount is %, false if fixed amount", 
      "discount": "number - Discount rate (%)",
      "calculated_amount": "number - Calculated discount amount"
    },
    "total_amount": "number - Final total amount"
  }
}

## Extraction Guidelines:
- **Supplier**: Look for business name, vendor name, or "Bill To" information
- **Payment Account**: Extract account numbers, card details, or payment method info
- **Payment Date**: Find transaction date, invoice date, or payment date
- **Payment Method**: Identify if paid by cash, card, check, bank transfer, etc.
- **Reference Number**: Look for invoice #, receipt #, transaction ID, or reference number
- **Tags**: Generate relevant tags based on the business type or expense category
- **Item Details**: Extract ALL line items from the document - create a separate object for each item/product/service listed
- **Categories**: Classify expenses (office supplies, travel, meals, equipment, etc.)
- **Calculations**: Verify that item totals sum to subtotal, and final calculations are accurate
- **Language Handling**: If you encounter Bangla/Bengali text, transliterate it into English letters (e.g., "টাকা" becomes "taka", "নাম" becomes "naam"). Do not translate the meaning, just write how the Bangla words sound in English.

## Important Notes:
- Return ONLY the JSON object, no additional text or explanations
- If information is missing, use null for strings and 0 for numbers
- Ensure all numbers are numeric values, not strings
- **Extract every single line item** - the item_details array should contain one object for each product/service listed in the document
- Double-check mathematical accuracy of totals and ensure all item totals sum to the subtotal
- VAT and tax should be considered same.
- If no tax/VAT or discount is present, set the respective amounts to 0

Now please analyze the provided image and extract the data according to this schema.
        """,
        "payslip_extraction": """
        This is a payslip/salary slip image. Please extract the following information in JSON format:
        {
          "employee_name": "",
          "employee_id": "",
          "pay_period": "",
          "gross_salary": "",
          "net_salary": "",
          "deductions": [],
          "company_name": ""
        }
        If any field is not found, use null.
        """,

        "structured_data": """
        Extract all structured data from this image and organize it in a clear, readable format. 
        Identify tables, forms, or any structured information and present it clearly.
        """,

        "key_value_pairs": """
        Extract all key-value pairs from this image. Present them as:
        Key: Value
        Key: Value
        etc.
        """,

        "cash_expense_extraction_V2": """
You are an expert document processing assistant. Your task is to extract structured data from invoice/receipt images and return the information in a specific JSON format.

## Instructions:
1. Analyze the provided image carefully to identify all relevant financial information
2. Extract the data according to the JSON schema provided below
3. If any required field is not visible or unclear in the image, use reasonable defaults or null values
4. For monetary amounts, use numbers without currency symbols
5. For dates, use the format found in the document or convert to YYYY-MM-DD if possible
6. Be precise with calculations - ensure totals match the extracted item details
7. The document may contain text in both English and Bangla (Bengali). For Bangla text, provide English transliteration (not translation) - write the Bangla words using English letters to show how they sound. For Bangla numbers convert them to
    English numerals (e.g., "১২৩" becomes "123").
## Required JSON Schema:
Return your response as a valid JSON object following this EXACT structure:

{
  "payment_details": {
    "supplier": "string - Name of the supplier/vendor",
    "payment_account": "string - Account used for payment",
    "payment_date": "string - Date of payment",
    "payment_method": "string - Mode of payment (cash, card, bank transfer, etc.)",
    "ref_no": "string - Reference/invoice number",
    "tags": ["array of relevant tags"]
  },
  "item_details": [
    {
      "category": "string - Expense category",
      "description": "string - Item description (optional)",
      "quantity": "number - Quantity of items",
      "unit_price": "number - Price per unit",
      "total": "number - Total amount for this item"
    },
    {
      "category": "string - Expense category for item 2",
      "description": "string - Item description (optional)",
      "quantity": "number - Quantity of items",
      "unit_price": "number - Price per unit", 
      "total": "number - Total amount for this item"
    }
    // ... additional items as found in the document
  ],
  "attachment": "string - File reference if mentioned",
  "memo": "string - Any additional notes or memo",
  "totals": {
    "sub_total": "number - Subtotal before tax and discount",
    "sales_tax": {
      "is_percentage": "boolean - true if tax/VAT is %, false if fixed amount",
      "tax": "number - Tax/VAT rate (%)", 
      "amount": "number - Calculated tax/VAT amount"
    },
    "discount": {
      "is_percentage": "boolean - true if discount is %, false if fixed amount", 
      "discount": "number - Discount rate (%)",
      "calculated_amount": "number - Calculated discount amount"
    },
    "total_amount": "number - Final total amount"
  }
}

## Extraction Guidelines:
- **Supplier**: Look for business name, vendor name, or "Bill To" information
- **Payment Account**: Extract account numbers, card details, or payment method info
- **Payment Date**: Find transaction date, invoice date, or payment date
- **Payment Method**: Identify if paid by cash, card, check, bank transfer, etc.
- **Reference Number**: Look for invoice #, receipt #, transaction ID, or reference number
- **Tags**: Generate relevant tags based on the business type or expense category
- **Item Details**: Extract ALL line items from the document - create a separate object for each item/product/service listed
- **Categories**: Classify expenses (office supplies, travel, meals, equipment, etc.)
- **Calculations**: Verify that item totals sum to subtotal, and final calculations are accurate
- **Language Handling**: If you encounter Bangla/Bengali text, transliterate it into English letters (e.g., "টাকা" becomes "taka", "নাম" becomes "naam"). Do not translate the meaning, just write how the Bangla words sound in English.
- **Tax/VAT Recognition**: Look for any tax-related terms including "Tax", "VAT", "Sales Tax", "Service Tax", "GST", or similar terms in both English and Bangla. Treat all of these as sales tax in the JSON structure.


## Important Notes:
- Return ONLY the JSON object, no additional text or explanations
- If information is missing, use null for strings and 0 for numbers
- Ensure all numbers are numeric values, not strings
- **Extract every single line item** - the item_details array should contain one object for each product/service listed in the document
- Double-check mathematical accuracy of totals and ensure all item totals sum to the subtotal
- VAT and tax should be considered same.
- If no tax/VAT or discount is present, set the respective amounts to 0

Now please analyze the provided image and extract the data according to this schema.
        """,
        
        
        
        "cash_expense_extraction_V3": """
You are an expert document processing assistant. Your task is to extract structured data from invoice/receipt images and return the information in a specific JSON format.

## Instructions:
1. Analyze the provided image carefully to identify all relevant financial information
2. Extract the data according to the JSON schema provided below
3. If any required field is not visible or unclear in the image, use reasonable defaults or null values
4. For monetary amounts, use numbers without currency symbols
5. For dates, use the format found in the document or convert to YYYY-MM-DD if possible
6. Be precise with calculations - ensure totals match the extracted item details
7. The document may contain text in both English and Bangla (Bengali). For Bangla text, provide English transliteration (not translation) - write the Bangla words using English letters to show how they sound. For Bangla numbers convert them to
    English numerals (e.g., "১২৩" becomes "123").  
8. Only extract information that is clearly visible in the image - never guess or infer missing data
9. If text is unclear or partially obscured, use null rather than making assumptions
10. Verify all mathematical calculations before finalizing the JSON response
## Required JSON Schema:
Return your response as a valid JSON object following this exact structure:

{
  "payment_details": {
    "supplier": "string - Name of the supplier/vendor",
    "payment_account": "string - Account used for payment",
    "payment_date": "string - Date of payment",
    "payment_method": "string - Mode of payment (cash, card, bank transfer, etc.)",
    "ref_no": "string - Reference/invoice number",
    "tags": ["array of relevant tags"]
  },
  "item_details": [
    {
      "category": "string - Expense category",
      "description": "string - Item description (optional)",
      "quantity": "number - Quantity of items",
      "unit_price": "number - Price per unit",
      "total": "number - Total amount for this item"
    },
    {
      "category": "string - Expense category for item 2",
      "description": "string - Item description (optional)",
      "quantity": "number - Quantity of items",
      "unit_price": "number - Price per unit", 
      "total": "number - Total amount for this item"
    }
    // ... additional items as found in the document
  ],
  "attachment": "string - File reference if mentioned",
  "memo": "string - Any additional notes or memo",
  "totals": {
    "sub_total": "number - Subtotal before tax and discount",
    "sales_tax": {
      "is_percentage": "boolean - true if tax/VAT is %, false if fixed amount",
      "tax": "number - Tax/VAT rate (%)", 
      "amount": "number - Calculated tax/VAT amount"
    },
    "discount": {
      "is_percentage": "boolean - true if discount is %, false if fixed amount", 
      "discount": "number - Discount rate (%)",
      "calculated_amount": "number - Calculated discount amount"
    },
    "total_amount": "number - Final total amount"
  }
}

## Extraction Guidelines:
- **Supplier**: Look for business name, vendor name, or "Bill To" information
- **Payment Account**: Extract account numbers, card details, or payment method info
- **Payment Date**: Find transaction date, invoice date, or payment date
- **Payment Method**: Identify if paid by cash, card, check, bank transfer, etc.
- **Reference Number**: Look for invoice #, receipt #, transaction ID, or reference number
- **Tags**: Generate relevant tags based on the business type or expense category
- **Item Details**: Extract ALL line items from the document - create a separate object for each item/product/service listed
- **Categories**: Classify expenses (office supplies, travel, meals, equipment, etc.)
- **Calculations**: Verify that item totals sum to subtotal, and final calculations are accurate
- **Language Handling**: If you encounter Bangla/Bengali text, transliterate it into English letters (e.g., "টাকা" becomes "taka", "নাম" becomes "naam"). Do not translate the meaning, just write how the Bangla words sound in English.
- **Tax/VAT Recognition**: Look for any tax-related terms including "Tax", "VAT", "Sales Tax", "Service Tax", "GST", or similar terms in both English and Bangla. Treat all of these as sales tax in the JSON structure.

## Validation Rules:
- **Mathematical Accuracy**: Ensure sum of all item totals equals subtotal
- **Date Formats**: Only use dates that are clearly visible; don't assume date formats
- **Number Precision**: Use exact numbers from the document, don't round unless specified
- **Text Clarity**: If you cannot read text clearly due to image quality, use null
- **Cross-Reference**: Verify supplier name appears consistently throughout the document
- **Currency Consistency**: Ensure all amounts use the same currency basis

## Important Notes:
- Return ONLY the JSON object, no additional text or explanations
- If information is missing, use null for strings and 0 for numbers
- Ensure all numbers are numeric values, not strings
- **Extract every single line item** - the item_details array should contain one object for each product/service listed in the document
- Double-check mathematical accuracy of totals and ensure all item totals sum to the subtotal
- VAT and tax should be considered same.
- If no tax/VAT or discount is present, set the respective amounts to 0
- Do not interpolate or guess missing information** - use null for unclear data
- Double-check all calculations manually** before submitting the JSON
- If image quality is poor and text is unreadable, explicitly use null values**
- Verify that reference numbers and dates match standard formats for the region**
- Cross-validate totals: item_totals → subtotal → (±tax/discount) → final_total**

## Final Check:
Before responding, verify:
1. All visible numbers are accurately transcribed
2. Mathematical relationships are correct
3. No information has been assumed or fabricated
4. Bangla text is transliterated, not translated
5. All required fields contain appropriate data types

Now please analyze the provided image and extract the data according to this schema.
        """,
        
                "cash_expense_extraction_V4": """
You are an expert document processing assistant. Extract structured data from invoice/receipt images and return as JSON.

CORE RULES:
- Extract only visible information - never guess missing data
- Use null for unclear/obscured text
- DO NOT hallucinate or invent information not present in the image
- If you cannot see text clearly, use null rather than guessing
- Convert Bangla numerals to English (১২৩ → 123)
- Transliterate Bangla text to English phonetics (টাকা → taka)
- Verify calculations match document totals
- Return ONLY the JSON object

JSON SCHEMA:
{
  "payment_details": {
    "supplier": "string - Name of the supplier/vendor",
    "payment_account": "string - Account used for payment",
    "payment_date": "string - Date of payment",
    "payment_method": "string - Mode of payment (cash, card, bank transfer, etc.)",
    "ref_no": "string - Reference/invoice number",
    "tags": ["array of relevant tags"]
  },
  "item_details": [
    {
      "category": "string - Expense category",
      "description": "string - Item description (optional)",
      "quantity": "number - Quantity of items",
      "unit_price": "number - Price per unit",
      "total": "number - Total amount for this item"
    },
    {
      "category": "string - Expense category for item 2",
      "description": "string - Item description (optional)",
      "quantity": "number - Quantity of items",
      "unit_price": "number - Price per unit", 
      "total": "number - Total amount for this item"
    }
    // ... additional items as found in the document
  ],
  "attachment": "string - File reference if mentioned",
  "memo": "string - Any additional notes or memo",
  "totals": {
    "sub_total": "number - Subtotal before tax and discount",
    "sales_tax": {
      "is_percentage": "boolean - true if tax/VAT is %, false if fixed amount",
      "tax": "number - Tax/VAT rate (%)", 
      "amount": "number - Calculated tax/VAT amount"
    },
    "discount": {
      "is_percentage": "boolean - true if discount is %, false if fixed amount", 
      "discount": "number - Discount rate (%)",
      "calculated_amount": "number - Calculated discount amount"
    },
    "total_amount": "number - Final total amount"
  }
}

EXTRACTION POINTS:
- Supplier: Business/vendor name
- Payment: Account, date, method (cash/card/transfer)
- Reference: Invoice/receipt/transaction number
- Items: Extract ALL line items as separate objects
- Categories: Classify expenses (office supplies, travel, meals, etc.)
- Tax/VAT: Include all tax variants (GST, Service Tax, etc.)
- Calculations: Verify item totals → subtotal → final total

VALIDATION:
- Sum of item totals = subtotal
- Use exact numbers from document
- Use null for missing/unclear data
- Use 0 for absent tax/discount amounts

Analyze the image and extract data according to this schema.
"""
    }


# Create sample prompts
sample_prompts = create_sample_prompts()

print("Sample prompts created:")
for name, prompt in sample_prompts.items():
    print(f"\n📝 {name.upper()}:")
    print(f"   {prompt[:1000]}...")

print("\nReady to use! Check the next cells for usage examples.")

Sample prompts created:

📝 GENERAL_OCR:
   Extract all text from this image and format it nicely. Preserve the structure and layout as much as possible....

📝 PAYSLIP_EXTRACTION:
   
        This is a payslip/salary slip image. Please extract the following information in JSON format:
        {
          "employee_name": "",
          "employee_id": "",
          "pay_period": "",
          "gross_salary": "",
          "net_salary": "",
          "deductions": [],
          "company_name": ""
        }
        If any field is not found, use null.
        ...

📝 STRUCTURED_DATA:
   
        Extract all structured data from this image and organize it in a clear, readable format. 
        Identify tables, forms, or any structured information and present it clearly.
        ...

📝 KEY_VALUE_PAIRS:
   
        Extract all key-value pairs from this image. Present them as:
        Key: Value
        Key: Value
        etc.
        ...

📝 CASH_EXPENSE_EXTRACTION:
   
You are an expert document

## OCR System Initialization

Initialize the OCR Benchmark system with the chosen AI model (default: Gemini) and verify image directories.

In [4]:
# Initialize the OCR Benchmark system
# By default, it uses Gemini model

try:
    # Initialize Default
    ocr = OCRBenchmark(model_name="huggingface", temperature=0.1)
    print("✅ OCR Benchmark system initialized successfully with Hugging Face!")
    print(f"Current model: {ocr.model_name}")
    
    # Check if we have images to process
    pay_slip_dir = "/home/tanjim_noor/Work/AI OCR BenchMark/Good Pictures"
    if os.path.exists(pay_slip_dir):
        image_files = ocr.get_images_from_directory(pay_slip_dir)
        print(f"Found {len(image_files)} image(s) in Pay Slip directory")
        if image_files:
            print("Sample images:")
            for i, img in enumerate(image_files):
                print(f"  {i+1}. {Path(img).name}")
        
except Exception as e:
    print(f"❌ Error initializing OCR system: {e}")
    print("Please make sure to set your API keys in the .env file")

✅ OCR Benchmark system initialized successfully with Hugging Face!
Current model: huggingface
Found 10 image(s) in Pay Slip directory
Sample images:
  1. IMG_20250826_154715.jpg
  2. IMG_20250826_154929.jpg
  3. IMG_20250826_155237.jpg
  4. IMG_20250826_155355.jpg
  5. IMG_20250826_155901.jpg
  6. IMG_20250826_160504.jpg
  7. IMG_20250826_160529.jpg
  8. IMG_20250826_160544.jpg
  9. IMG_20250826_160721.jpg
  10. IMG_20250826_160928.jpg


## Single Image Processing Example

Demonstrate processing a single image from the directory using the cash expense extraction prompt.

In [5]:
# Example 1: Process a single image
# Let's try to process one image from the Pay Slip directory

try:
    pay_slip_dir = "/home/tanjim_noor/Work/AI OCR BenchMark/Good Pictures"
    if os.path.exists(pay_slip_dir):
        image_files = ocr.get_images_from_directory(pay_slip_dir)
        
        if image_files:
            # Process the first image with payslip extraction prompt
            first_image = image_files[0]
            print(f"Processing: {Path(first_image).name}")
            print("Using cash OCR prompt...")

            result = ocr.extract_data_from_image(first_image, sample_prompts["cash_expense_extraction_V2"])
            
            if result["success"]:
                print("\n✅ SUCCESS!")
                print(f"Model used: {result['model_used']}")
                print(f"Extracted data:\n{result['extracted_data']}")
            else:
                print(f"\n❌ FAILED: {result.get('error', 'Unknown error')}")
        else:
            print("No images found in the directory")
    else:
        print("Pay Slip directory not found")
        
except Exception as e:
    print(f"Error: {e}")
    print("Make sure your API key is set correctly in the .env file")

Processing: IMG_20250826_154715.jpg
Using cash OCR prompt...
🚀 Starting image processing...
🔍 File extension: .jpg → MIME type: image/jpeg
🖼️ Optimizing image...
📏 Original file size: 4.31 MB
📐 Original dimensions: 4624x3472
🔧 HuggingFace aggressive resize by ratio 0.5: 2312x1736
💾 HuggingFace JPEG quality: 50
✅ HuggingFace optimized size: 0.35 MB
📝 Encoding to base64...
📊 Base64 size: 0.46 MB
🤖 Sending to HUGGINGFACE...
⏱️ API call completed in 14.53 seconds

✅ SUCCESS!
Model used: huggingface
Extracted data:
```json
{
  "payment_details": {
    "supplier": "ACI Logistics Limited",
    "payment_account": null,
    "payment_date": "26/06/2025",
    "payment_method": "City Bank",
    "ref_no": "L54008",
    "tags": ["retail", "grocery", "food"]
  },
  "item_details": [
    {
      "category": "grocery",
      "description": "Fresh Soyabean Oil 5ltr",
      "quantity": 1.00,
      "unit_price": 922.00,
      "total": 922.00
    },
    {
      "category": "grocery",
      "description": "

## Batch Processing - Process All Images

This section demonstrates batch processing of all images in a directory, saving results to JSON files with enhanced formatting.

In [6]:
# Example 2: Process all images in directory (Batch Processing)
# This will process all images in the Pay Slip directory

# Uncomment the following code to process all images:

try:
    pay_slip_dir = "/home/tanjim_noor/Work/AI OCR BenchMark/For benchmark"
    output_file = "/home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output/ocr_result_qwen_1.json"
    
    print("Starting batch processing...")
    print(f"Input directory: {pay_slip_dir}")
    print(f"Output file: {output_file}")
    print("=" * 50)
    
    # Process all images with payslip extraction prompt
    results = ocr.process_directory(
        directory_path=pay_slip_dir,
        prompt=sample_prompts["cash_expense_extraction_V2"],
        output_file=output_file
    )
    
    # Summary
    successful = sum(1 for r in results if r["success"])
    failed = len(results) - successful
    
    print("=" * 50)
    print(f"📊 PROCESSING SUMMARY:")
    print(f"Total images: {len(results)}")
    print(f"Successful: {successful}")
    print(f"Failed: {failed}")
    print(f"Results saved to: {output_file}")
    
except Exception as e:
    print(f"Error during batch processing: {e}")


print("Batch processing code is commented out.")
#print("Uncomment the code above to process all images in the directory.")
#print("Note: This will use API calls for each image, so be mindful of costs!")

Starting batch processing...
Input directory: /home/tanjim_noor/Work/AI OCR BenchMark/For benchmark
Output file: /home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output/ocr_result_qwen_1.json
Found 10 image(s) to process...
Processing image 1/10: IMG_20250826_154308~2.jpg
🚀 Starting image processing...
🔍 File extension: .jpg → MIME type: image/jpeg
🖼️ Optimizing image...
📏 Original file size: 1.33 MB
📐 Original dimensions: 3472x4624
🔧 HuggingFace aggressive resize by ratio 0.5: 1736x2312
💾 HuggingFace JPEG quality: 50
✅ HuggingFace optimized size: 0.17 MB
📝 Encoding to base64...
📊 Base64 size: 0.23 MB
🤖 Sending to HUGGINGFACE...
⏱️ API call completed in 11.79 seconds
✅ Successfully processed IMG_20250826_154308~2.jpg
📊 Extracted structured data format: json
Processing image 2/10: IMG_20250826_154831.jpg
🚀 Starting image processing...
🔍 File extension: .jpg → MIME type: image/jpeg
🖼️ Optimizing image...
📏 Original file size: 4.62 MB
📐 Original dimensions: 4624x3472
🔧 HuggingFace aggressive r

## Test Data Samples

Sample JSON data structures for testing and validation purposes, including both predicted output and actual ground truth data.

In [ ]:
output_json = {
    "payment_details": {
        "supplier": "Maa Fruits & Departmental Store",
        "payment_account": None,
        "payment_date": "2020-06-20",
        "payment_method": "Cash",
        "ref_no": None,
        "tags": [
            "Fruits",
            "Departmental Store",
            "Retail",
            "Wholesale"
        ]
    },
    "item_details": [
        {
            "category": "Fruits",
            "description": "Tomatoes",
            "quantity": 80,
            "unit_price": 7.5,
            "total": 600
        }
    ],
    "attachment": None,
    "memo": "Soles goods are not returnable. Welcome come again.",
    "totals": {
        "sub_total": 600,
        "sales_tax": {
            "is_percentage": False,
            "tax": 0,
            "amount": 0
        },
        "discount": {
            "is_percentage": False,
            "discount": 0,
            "calculated_amount": 0
        },
        "total_amount": 600
    }
}


actual_json = {
    "payment_details": {
        "supplier": "Maa Fruits & Departmental Store",
        "payment_account": None,
        "payment_date": "2020-06-20",
        "payment_method": "Cash",
        "ref_no": None,
        "tags": [
            "Fruits",
            "Departmental Store",
            "Retail",
            "Wholesale"
        ]
    },
    "item_details": [
        {
            "category": "Fruits",
            "description": "Tomatoes",
            "quantity": 80,
            "unit_price": 10,
            "total": 800
        }
    ],
    "attachment": None,
    "memo": "Soled goods are not returnable. Welcome come again.",
    "totals": {
        "sub_total": 800,
        "sales_tax": {
            "is_percentage": False,
            "tax": 0,
            "amount": 0
        },
        "discount": {
            "is_percentage": False,
            "discount": 0,
            "calculated_amount": 0
        },
        "total_amount": 800
    }
}



output_json_2 = {
  "payment_details": {
    "supplier": "SHWAPNO (ACI Logistics Limited)",
    "payment_account": "City Bank",
    "payment_date": "2025-06-26",
    "payment_method": "Bank Transfer/Card (implied by City Bank)",
    "ref_no": "D0612506260068",
    "tags": ["Groceries", "Supermarket", "Retail", "Food"]
  },
  "item_details": [
    {
      "category": "Groceries",
      "description": "Beef Premium Cube kg",
      "quantity": 6.33,
      "unit_price": 775.00,
      "total": 4901.88
    },
    {
      "category": "Groceries",
      "description": "Broiler Chicken Breast Bon",
      "quantity": 0.50,
      "unit_price": 534.00,
      "total": 267.00
    },
    {
      "category": "Groceries",
      "description": "Capsicum Green kg",
      "quantity": 1.01,
      "unit_price": 320.00,
      "total": 321.60
    },
    {
      "category": "Groceries",
      "description": "Carrot China (China Gajor) K",
      "quantity": 1.40,
      "unit_price": 160.00,
      "total": 224.00
    },
    {
      "category": "Groceries",
      "description": "Cucumber (Shosha) kg",
      "quantity": 2.69,
      "unit_price": 55.00,
      "total": 147.68
    },
    {
      "category": "Groceries",
      "description": "Green Chili (Kacha Morich)",
      "quantity": 0.67,
      "unit_price": 100.00,
      "total": 66.50
    },
    {
      "category": "Groceries",
      "description": "Green Papaya (Kacha Pepe)",
      "quantity": 3.00,
      "unit_price": 45.00,
      "total": 134.78
    },
    {
      "category": "Groceries",
      "description": "Lemon Long (Lomba Lebu) PC",
      "quantity": 8.00,
      "unit_price": 6.00,
      "total": 48.00
    },
    {
      "category": "Groceries",
      "description": "Long Bean (Boroboti) KG",
      "quantity": 1.83,
      "unit_price": 90.00,
      "total": 164.70
    },
    {
      "category": "Groceries",
      "description": "Piyaj Deshi Loose kg",
      "quantity": 2.66,
      "unit_price": 58.00,
      "total": 154.28
    },
    {
      "category": "Groceries",
      "description": "Shonalika Dressed Classic",
      "quantity": 1.65,
      "unit_price": 667.00,
      "total": 1097.22
    },
    {
      "category": "Groceries",
      "description": "ACI Pure Corn Flour 150g",
      "quantity": 1.00,
      "unit_price": 60.00,
      "total": 60.00
    },
    {
      "category": "Groceries",
      "description": "Shwapno Black Pepper Powde",
      "quantity": 1.00,
      "unit_price": 129.00,
      "total": 129.00
    }
  ],
  "attachment": False,
  "memo": "Thank you for shopping with SHWAPNO. Please visit www.shwapno.com for home delivery. Purchase of defected item must be exchanged by 24 hours with invoice. For any queries, suggestions or complaints, please call 16469 (9:00 AM - 6:00 PM). Earned points will expire within 6 months from the date of the transaction if not redeemed. VAT against this challan is payable through central registration. Prices inclusive of standard VAT except exempted items, VAT Payable TK. : 7.17.",
  "totals": {
    "sub_total": 7716.64,
    "sales_tax": {
      "is_percentage": False,
      "tax": 0,
      "amount": 0
    },
    "discount": {
      "is_percentage": False,
      "discount": 0,
      "calculated_amount": 0
    },
    "total_amount": 7716.64
  }
}

actual_json_2 = {"payment_details": {
    "supplier": "SHWAPNO (ACI Logistics Limited)",
    "payment_account": "City Bank",
    "payment_date": "2025-06-26",
    "payment_method": "Bank Transfer/Card (implied by City Bank)",
    "ref_no": "D0612506260068",
    "tags": ["Groceries", "Supermarket", "Retail", "Food"]
  },
  "item_details": [
    {
      "category": "Groceries",
      "description": "Beef Premium Cube kg",
      "quantity": 6.33,
      "unit_price": 775.00,
      "total": 4901.88
    },
    {
      "category": "Groceries",
      "description": "Broiler Chicken Breast Bon",
      "quantity": 0.50,
      "unit_price": 534.00,
      "total": 267.00
    },
    {
      "category": "Groceries",
      "description": "Capsicum Green kg",
      "quantity": 1.01,
      "unit_price": 320.00,
      "total": 321.60
    },
    {
      "category": "Groceries",
      "description": "Carrot China (China Gajor) K",
      "quantity": 1.40,
      "unit_price": 160.00,
      "total": 224.00
    },
    {
      "category": "Groceries",
      "description": "Cucumber (Shosha) kg",
      "quantity": 2.69,
      "unit_price": 55.00,
      "total": 147.68
    },
    {
      "category": "Groceries",
      "description": "Green Chili (Kacha Morich)",
      "quantity": 0.67,
      "unit_price": 100.00,
      "total": 66.50
    },
    {
      "category": "Groceries",
      "description": "Green Papaya (Kacha Pepe)",
      "quantity": 3.00,
      "unit_price": 45.00,
      "total": 134.78
    },
    {
      "category": "Groceries",
      "description": "Lemon Long (Lomba Lebu) PC",
      "quantity": 8.00,
      "unit_price": 6.00,
      "total": 48.00
    },
    {
      "category": "Groceries",
      "description": "Long Bean (Boroboti) KG",
      "quantity": 1.83,
      "unit_price": 90.00,
      "total": 164.70
    },
    {
      "category": "Groceries",
      "description": "Piyaj Deshi Loose kg",
      "quantity": 2.66,
      "unit_price": 58.00,
      "total": 154.28
    },
    {
      "category": "Groceries",
      "description": "Shonalika Dressed Classic",
      "quantity": 1.65,
      "unit_price": 667.00,
      "total": 1097.22
    },
    {
      "category": "Groceries",
      "description": "ACI Pure Corn Flour 150g",
      "quantity": 1.00,
      "unit_price": 60.00,
      "total": 60.00
    },
    {
      "category": "Groceries",
      "description": "Shwapno Black Pepper Powde",
      "quantity": 1.00,
      "unit_price": 129.00,
      "total": 129.00
    }
  ],
  "attachment": False,
  "memo": "Thank you for shopping with SHWAPNO. Please visit www.shwapno.com for home delivery. Purchase of defected item must be exchanged by 24 hours with invoice. For any queries, suggestions or complaints, please call 16469 (9:00 AM - 6:00 PM). Earned points will expire within 6 months from the date of the transaction if not redeemed. VAT against this challan is payable through central registration. Prices inclusive of standard VAT except exempted items, VAT Payable TK. : 7.17.",
  "totals": {
    "sub_total": 7716.64,
    "sales_tax": {
      "is_percentage": False,
      "tax": 0,
      "amount": 7.17
    },
    "discount": {
      "is_percentage": False,
      "discount": 0,
      "calculated_amount": 30.80
    },
    "total_amount": 7686.00
  }
}




## Install Additional Dependencies

Install sentence-transformers library for semantic similarity scoring in the benchmark system.

In [ ]:
# %pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 5.6 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 12.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 13.1 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 10.4 MB/s  0:01:13m0:00:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 7.9 MB/s  0:01:11m0:00:0100:03m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 12.8 MB/s  0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 16.3 MB/s  0:00:05m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 45.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 11.7 MB/s  0:00:58m0:00:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 10.9 MB/s  0:00:17m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16

## 📊 Scoring System Implementation

Advanced scoring system to evaluate OCR extraction accuracy using multiple comparison methods including exact matching, numeric tolerance, semantic similarity, and embedding-based similarity.

In [6]:
import math
from typing import Any, Dict, List
from difflib import SequenceMatcher
import numpy as np

# Optional: install if not already
# pip install sentence-transformers

from sentence_transformers import SentenceTransformer

# Load a small model once (fast + lightweight)
embed_model = SentenceTransformer("all-MiniLM-L6-v2")


# --- helpers ---
def get_by_path(data: dict, path: str):
    """
    Supports:
    - normal dot paths ("totals.sub_total")
    - indexed paths ("item_details[0].quantity")
    - wildcard array paths ("item_details.quantity") -> returns list of values
    """
    keys = path.replace("]", "").split(".")
    val = data
    for k in keys:
        if isinstance(val, list):
            # Apply to each element if list
            if k.isdigit():
                val = val[int(k)]
            else:
                val = [v[k] for v in val]
        else:
            if "[" in k:  # explicit index
                field, idx = k.split("[")
                val = val[field][int(idx)]
            else:
                val = val[k]
    return val


# --- comparators ---
def exact_match(a, b) -> float:
    return 1.0 if str(a) == str(b) else 0.0

def numeric_tolerance(a, b, tolerance: float = 0.1) -> float:
    try:
        a, b = float(a), float(b)
    except (ValueError, TypeError):
        return 0.0
    if a == b:
        return 1.0
    rel_error = abs(a - b) / (abs(b) + 1e-8)
    return max(0.0, 1 - rel_error / tolerance)

def semantic_similarity(a, b) -> float:
    """Cheap semantic: edit similarity."""
    if not a or not b:
        return 0.0
    return SequenceMatcher(None, str(a).lower(), str(b).lower()).ratio()

def embedding_similarity(a, b) -> float:
    """Semantic similarity using embeddings + cosine similarity."""
    if not a or not b:
        return 0.0
    emb_a = embed_model.encode(str(a), convert_to_numpy=True)
    emb_b = embed_model.encode(str(b), convert_to_numpy=True)
    cos_sim = np.dot(emb_a, emb_b) / (np.linalg.norm(emb_a) * np.linalg.norm(emb_b))
    # map cosine [-1,1] to [0,1]
    return float((cos_sim + 1) / 2)


COMPARATORS = {
    "exact": exact_match,
    "numeric": numeric_tolerance,
    "semantic": semantic_similarity,   # lightweight string-based
    "embedding": embedding_similarity, # heavy but meaningful
}


# --- main scoring ---
def score_json(pred: dict, actual: dict, config: List[Dict[str, Any]]) -> Dict[str, Any]:
    total_weight = sum(field["weight"] for field in config)
    breakdown = []
    weighted_sum = 0.0

    for field in config:
        path = field["path"]
        weight = field.get("weight", 1.0)
        comparator = COMPARATORS[field["comparator"]]
        tolerance = field.get("tolerance", 0.1)

        pred_val = get_by_path(pred, path)
        actual_val = get_by_path(actual, path)

        # Case 1: scalar
        if not isinstance(pred_val, list):
            pred_val, actual_val = [pred_val], [actual_val]

        # If lengths differ, penalize missing items
        max_len = max(len(pred_val), len(actual_val))
        scores = []
        for i in range(max_len):
            pv = pred_val[i] if i < len(pred_val) else None
            av = actual_val[i] if i < len(actual_val) else None

            if field["comparator"] == "numeric":
                s = comparator(pv, av, tolerance)
            else:
                s = comparator(pv, av)
            scores.append(s)

            breakdown.append({
                "path": f"{path}[{i}]",
                "pred": pv,
                "actual": av,
                "comparator": field["comparator"],
                "score": round(s, 3),
                "weight": weight,
                "weighted": round(weight * s, 3)
            })

        avg_score = sum(scores) / max_len if max_len > 0 else 0
        weighted_sum += weight * avg_score

    final_score = weighted_sum / total_weight if total_weight > 0 else 0
    return {
        "final_score": round(final_score, 3),
        "breakdown": breakdown
    }


## ⚙️ Scoring Configuration and Testing

Configure the scoring system with field-specific weights and comparison methods, then test it with sample data.

In [15]:
config = [
    {"path": "payment_details.supplier", "weight": 1, "comparator": "exact"},
    {"path": "payment_details.payment_date", "weight": 1, "comparator": "exact"},
    {"path": "payment_details.payment_account", "weight": 1, "comparator": "embedding"},
    {"path": "payment_details.payment_method", "weight": 1, "comparator": "exact"},
    {"path": "payment_details.ref_no", "weight": 1, "comparator": "exact"},
    {"path": "payment_details.tags", "weight": 1, "comparator": "semantic"},
    {"path": "item_details.category", "weight": 1, "comparator": "exact"},
    {"path": "item_details.description", "weight": 1, "comparator": "embedding"},
    {"path": "item_details.quantity", "weight": 1, "comparator": "numeric", "tolerance": 0.05},
    {"path": "item_details.unit_price", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
    {"path": "item_details.total", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
    {"path": "attachment", "weight": 1, "comparator": "exact"},
    {"path": "memo", "weight": 1, "comparator": "embedding"},
    {"path": "totals.sub_total", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
    {"path": "totals.sales_tax.is_percentage", "weight": 1, "comparator": "exact"},
    {"path": "totals.sales_tax.tax", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
    {"path": "totals.sales_tax.amount", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
    {"path": "totals.discount.is_percentage", "weight": 1, "comparator": "exact"},
    {"path": "totals.discount.discount", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
    {"path": "totals.discount.calculated_amount", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
    {"path": "totals.total_amount", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
]


# result = score_json(output_json_2, actual_json_2, config)

# print("Final Score:", result["final_score"])
# for r in result["breakdown"]:
#     print(r)

## 📈 Batch Scoring and Performance Analysis

Implement and execute batch scoring functionality to evaluate OCR performance across multiple images, comparing results with ground truth data.

In [16]:
# Batch scoring for all images in JSON files
import json
import os

def score_all_images(results_json_path, actual_json_path, config):
    """
    Score all images by comparing results JSON with actual JSON
    
    Args:
        results_json_path: Path to the OCR results JSON file
        actual_json_path: Path to the actual/ground truth JSON file
        config: Configuration list for scoring (same as above)
    
    Returns:
        Dict with overall statistics and individual image scores
    """
    
    # Load JSON files
    try:
        with open(results_json_path, 'r') as f:
            results_data = json.load(f)
        with open(actual_json_path, 'r') as f:
            actual_data = json.load(f)
    except FileNotFoundError as e:
        print(f"Error: File not found - {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON - {e}")
        return None
    
    # Create lookup dictionary for actual data by image path
    actual_lookup = {item['image_path']: item for item in actual_data}
    
    # Score each image
    image_scores = []
    total_score = 0
    processed_count = 0
    
    print("=" * 80)
    print("BATCH SCORING RESULTS")
    print("=" * 80)
    
    for result_item in results_data:
        image_path = result_item['image_path']
        image_name = os.path.basename(image_path)
        
        # Find corresponding actual data
        if image_path not in actual_lookup:
            print(f"❌ No ground truth found for: {image_name}")
            continue
            
        actual_item = actual_lookup[image_path]
        
        # Extract structured data for comparison
        result_structured = result_item.get('structured_data', {})
        actual_structured = actual_item.get('structured_data', {})
        
        # Score this image
        if result_structured and actual_structured:
            score_result = score_json(result_structured, actual_structured, config)
            final_score = score_result['final_score']
            
            image_scores.append({
                'image_name': image_name,
                'image_path': image_path,
                'score': final_score,
                'breakdown': score_result['breakdown']
            })
            
            total_score += final_score
            processed_count += 1
            
            # Print score for this image
            print(f"📊 {image_name}: {final_score:.3f}")
            
        else:
            print(f"⚠️  Missing structured data for: {image_name}")
    
    # Calculate overall statistics
    if processed_count > 0:
        average_score = total_score / processed_count
        
        print("\n" + "=" * 80)
        print("SUMMARY STATISTICS")
        print("=" * 80)
        print(f"📈 Total Images Processed: {processed_count}")
        print(f"📊 Average Score: {average_score:.3f}")
        print(f"🎯 Total Score Sum: {total_score:.3f}")
        print(f"📉 Lowest Score: {min(item['score'] for item in image_scores):.3f}")
        print(f"📈 Highest Score: {max(item['score'] for item in image_scores):.3f}")
        
        # Show breakdown of scores
        score_ranges = {'0.0-0.2': 0, '0.2-0.4': 0, '0.4-0.6': 0, '0.6-0.8': 0, '0.8-1.0': 0}
        for item in image_scores:
            score = item['score']
            if score < 0.2:
                score_ranges['0.0-0.2'] += 1
            elif score < 0.4:
                score_ranges['0.2-0.4'] += 1
            elif score < 0.6:
                score_ranges['0.4-0.6'] += 1
            elif score < 0.8:
                score_ranges['0.6-0.8'] += 1
            else:
                score_ranges['0.8-1.0'] += 1
        
        print("\n📊 Score Distribution:")
        for range_name, count in score_ranges.items():
            print(f"   {range_name}: {count} images")
            
    else:
        average_score = 0
        print("❌ No images were processed successfully")
    
    return {
        'processed_count': processed_count,
        'average_score': average_score,
        'total_score': total_score,
        'image_scores': image_scores
    }

# Example usage:
# Set the paths to your JSON files
results_json_path = "/home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output/ocr_results_cash_expense_v2_huggingface_20250902_144246.json"
actual_json_path = "/home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output/ocr_qwen_actual_1_structured_only.json"

# Use the same config as defined above
batch_results = score_all_images(results_json_path, actual_json_path, config)

# Optionally, save detailed results to a file
if batch_results:
    output_path = "/home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output/batch_scoring_results_qwen.json"
    with open(output_path, 'w') as f:
        json.dump(batch_results, f, indent=2)
    print(f"\n💾 Detailed results saved to: {output_path}")

BATCH SCORING RESULTS
📊 IMG_20250826_154308~2.jpg: 0.815
❌ No ground truth found for: IMG_20250826_154831.jpg
⚠️  Missing structured data for: IMG_20250826_154929.jpg
📊 IMG_20250826_155237.jpg: 0.582
📊 IMG_20250826_155754.jpg: 0.750
📊 IMG_20250826_160041.jpg: 0.549
📊 IMG_20250826_160529.jpg: 0.495
📊 IMG_20250826_160644.jpg: 0.588
📊 IMG_20250826_160928.jpg: 0.438
📊 IMG_20250826_160949.jpg: 0.492

SUMMARY STATISTICS
📈 Total Images Processed: 8
📊 Average Score: 0.589
🎯 Total Score Sum: 4.709
📉 Lowest Score: 0.438
📈 Highest Score: 0.815

📊 Score Distribution:
   0.0-0.2: 0 images
   0.2-0.4: 0 images
   0.4-0.6: 6 images
   0.6-0.8: 1 images
   0.8-1.0: 1 images

💾 Detailed results saved to: /home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output/batch_scoring_results_qwen.json


### Additional Examples and Utilities

This section contains additional examples including batch processing templates, model switching, custom prompts, and benchmarking utilities.

In [ ]:
# Example 2: Process all images in directory (Batch Processing)
# This will process all images in the Pay Slip directory

# Uncomment the following code to process all images:
"""
try:
    pay_slip_dir = "/home/tanjim_noor/Work/AI OCR BenchMark/Pay Slip"
    output_file = "/home/tanjim_noor/Work/AI OCR BenchMark/ocr_results.json"
    
    print("Starting batch processing...")
    print(f"Input directory: {pay_slip_dir}")
    print(f"Output file: {output_file}")
    print("=" * 50)
    
    # Process all images with payslip extraction prompt
    results = ocr.process_directory(
        directory_path=pay_slip_dir,
        prompt=sample_prompts["payslip_extraction"],
        output_file=output_file
    )
    
    # Summary
    successful = sum(1 for r in results if r["success"])
    failed = len(results) - successful
    
    print("=" * 50)
    print(f"📊 PROCESSING SUMMARY:")
    print(f"Total images: {len(results)}")
    print(f"Successful: {successful}")
    print(f"Failed: {failed}")
    print(f"Results saved to: {output_file}")
    
except Exception as e:
    print(f"Error during batch processing: {e}")
"""

print("Batch processing code is commented out.")
print("Uncomment the code above to process all images in the directory.")
print("Note: This will use API calls for each image, so be mindful of costs!")

### Model Switching Examples

In [ ]:
# Example 3: Switch between different models
# Demonstrate model-agnostic capabilities

print("🔄 Model Switching Examples")
print("=" * 40)

# Current model
print(f"Current model: {ocr.model_name}")

print("\n📝 Available models:")
print("1. gemini - Google Gemini 1.5 Flash")
print("2. openai - OpenAI GPT-4 Vision")  
print("3. anthropic - Claude 3.5 Sonnet")

print("\n🔧 How to switch models:")
print("ocr.change_model('openai')    # Switch to OpenAI")
print("ocr.change_model('anthropic') # Switch to Claude")
print("ocr.change_model('gemini')    # Switch to Gemini")

print("\n⚠️ Note: Make sure to set the corresponding API keys in .env file")

# Example of switching (commented out)
"""
# Switch to OpenAI
try:
    ocr.change_model('openai')
    # Process an image with OpenAI
    result = ocr.extract_data_from_image(image_path, prompt)
except Exception as e:
    print(f"OpenAI error: {e}")

# Switch to Claude
try:
    ocr.change_model('anthropic')
    # Process an image with Claude
    result = ocr.extract_data_from_image(image_path, prompt)
except Exception as e:
    print(f"Claude error: {e}")
"""

### Custom Prompts and Utilities

In [14]:
# Example 4: Custom Prompts and Utilities

def create_custom_prompt(task_type: str, specific_fields: Optional[List[str]] = None) -> str:
    """
    Helper function to create custom prompts for different OCR tasks.
    
    Args:
        task_type: Type of document ('invoice', 'receipt', 'form', 'table', 'general')
        specific_fields: List of specific fields to extract
    """
    base_prompts = {
        'invoice': "Extract invoice information including invoice number, date, vendor, amount, tax, and line items.",
        'receipt': "Extract receipt information including store name, date, items purchased, prices, and total amount.",
        'form': "Extract all form fields and their values from this document.",
        'table': "Extract table data and organize it in a structured format with headers and rows.",
        'general': "Extract all text content from this image, preserving structure and formatting."
    }
    
    prompt = base_prompts.get(task_type, base_prompts['general'])
    
    if specific_fields:
        fields_str = ", ".join(specific_fields)
        prompt += f" Focus specifically on extracting: {fields_str}."
    
    prompt += " Present the information in a clear, structured format."
    return prompt

# Example custom prompts
print("🎯 Custom Prompt Examples")
print("=" * 30)

# Create some custom prompts
custom_prompts = {
    "payslip_detailed": create_custom_prompt("form", [
        "employee name", "employee ID", "salary period", 
        "gross pay", "net pay", "tax deductions", "company name"
    ]),
    
    "receipt_analysis": create_custom_prompt("receipt", [
        "store name", "purchase date", "items", "total amount", "payment method"
    ]),
    
    "table_extraction": create_custom_prompt("table"),
    
    "general_ocr": create_custom_prompt("general")
}

for name, prompt in custom_prompts.items():
    print(f"\n📝 {name.upper()}:")
    print(f"   {prompt}")

print("\n✨ You can use these prompts with:")
print("   result = ocr.extract_data_from_image(image_path, custom_prompts['payslip_detailed'])")

🎯 Custom Prompt Examples

📝 PAYSLIP_DETAILED:
   Extract all form fields and their values from this document. Focus specifically on extracting: employee name, employee ID, salary period, gross pay, net pay, tax deductions, company name. Present the information in a clear, structured format.

📝 RECEIPT_ANALYSIS:
   Extract receipt information including store name, date, items purchased, prices, and total amount. Focus specifically on extracting: store name, purchase date, items, total amount, payment method. Present the information in a clear, structured format.

📝 TABLE_EXTRACTION:
   Extract table data and organize it in a structured format with headers and rows. Present the information in a clear, structured format.

📝 GENERAL_OCR:
   Extract all text content from this image, preserving structure and formatting. Present the information in a clear, structured format.

✨ You can use these prompts with:
   result = ocr.extract_data_from_image(image_path, custom_prompts['payslip_detailed

### 🏆 Benchmarking Multiple Models

In [ ]:
# Example 5: Benchmarking Multiple Models
# Compare results from different models on the same image

def benchmark_models(image_path: str, prompt: str, models: Optional[List[str]] = None) -> Dict:
    """
    Benchmark multiple models on the same image and prompt.
    
    Args:
        image_path: Path to the image file
        prompt: Prompt to use for extraction
        models: List of models to test (defaults to available models)
        
    Returns:
        Dictionary with results from each model
    """
    if models is None:
        models = ["gemini", "openai", "anthropic"]
    
    benchmark_results = {
        "image_path": image_path,
        "prompt": prompt,
        "timestamp": datetime.now().isoformat(),
        "results": {}
    }
    
    for model_name in models:
        print(f"Testing {model_name}...")
        try:
            # Create new OCR instance for each model
            test_ocr = OCRBenchmark(model_name=model_name, temperature=0.1)
            
            # Check if model was initialized successfully
            if test_ocr.model is None:
                print(f"⚠️ {model_name} - Skipped: API key not available")
                benchmark_results["results"][model_name] = {
                    "success": False,
                    "error": f"API key not available for {model_name}",
                    "model_used": model_name
                }
                continue
            
            result = test_ocr.extract_data_from_image(image_path, prompt)
            benchmark_results["results"][model_name] = result
            
            if result["success"]:
                print(f"✅ {model_name} - Success")
            else:
                print(f"❌ {model_name} - Failed: {result.get('error', 'Unknown error')}")
                
        except Exception as e:
            print(f"❌ {model_name} - Error: {str(e)}")
            benchmark_results["results"][model_name] = {
                "success": False,
                "error": str(e),
                "model_used": model_name
            }
    
    return benchmark_results

# Utility function to compare and analyze results
def analyze_benchmark_results(benchmark_results: Dict):
    """Analyze and display benchmark results."""
    print("\n📊 BENCHMARK ANALYSIS")
    print("=" * 50)
    
    results = benchmark_results["results"]
    successful_models = [model for model, result in results.items() if result.get("success", False)]
    failed_models = [model for model, result in results.items() if not result.get("success", False)]
    
    print(f"Image: {Path(benchmark_results['image_path']).name}")
    print(f"Total models tested: {len(results)}")
    print(f"Successful: {len(successful_models)} - {successful_models}")
    print(f"Failed: {len(failed_models)} - {failed_models}")
    
    if successful_models:
        print("\n📝 Extracted Data Comparison:")
        for model in successful_models:
            print(f"\n🤖 {model.upper()}:")
            extracted_data = results[model]["extracted_data"]
            # Show first 200 characters
            preview = extracted_data[:200] + "..." if len(extracted_data) > 200 else extracted_data
            print(f"   {preview}")

print("🚀 Benchmarking utilities defined!")
print("Use benchmark_models() to compare multiple models on the same image.")

# Example usage (commented out):
"""
# Benchmark all models on a single image
image_path = "/path/to/your/image.png"
prompt = sample_prompts["payslip_extraction"]

benchmark_results = benchmark_models(image_path, prompt)
analyze_benchmark_results(benchmark_results)

# Save benchmark results
with open("benchmark_results.json", "w") as f:
    json.dump(benchmark_results, f, indent=2)
"""

# Quick Start Guide

## Setup Instructions

1. **Set up API Keys** - Add your API keys to the `.env` file:
   ```bash
   GOOGLE_API_KEY=your_actual_google_api_key
   OPENAI_API_KEY=your_actual_openai_api_key  
   ANTHROPIC_API_KEY=your_actual_anthropic_api_key
   ```

2. **Initialize the OCR System**:
   ```python
   ocr = OCRBenchmark(model_name="gemini")  # Default: Gemini
   ```

3. **Process a Single Image**:
   ```python
   result = ocr.extract_data_from_image("path/to/image.png", "Extract all text")
   ```

4. **Process All Images in Directory**:
   ```python
   results = ocr.process_directory("path/to/directory", "Extract payslip data")
   ```

## Available Models
- **gemini** - Google Gemini 1.5 Flash (Default)
- **openai** - OpenAI GPT-4 Vision  
- **anthropic** - Claude 3.5 Sonnet

## Features
✅ **Model Agnostic** - Switch between models easily  
✅ **Batch Processing** - Process entire directories  
✅ **Custom Prompts** - Flexible prompt engineering  
✅ **JSON Output** - Save results to files  
✅ **Error Handling** - Robust error management  
✅ **Benchmarking** - Compare model performance  

## Next Steps
1. Set your API keys in `.env`
2. Run the initialization cell
3. Try processing a single image first
4. Experiment with different prompts
5. Compare results across models

### API Keys Status Check

Verify which AI model API keys are available and properly configured in the environment.

In [ ]:
# Check API Keys Availability
print("🔑 API Keys Status Check")
print("=" * 30)

# Create a temporary OCR instance to check API keys
temp_ocr = OCRBenchmark()
api_status = temp_ocr.check_api_keys()

print("Available models:")
for model, available in api_status.items():
    status = "✅ Available" if available else "❌ Missing API Key"
    print(f"  {model}: {status}")

print(f"\nRecommended model: {next((model for model, available in api_status.items() if available), 'None available')}")

if not any(api_status.values()):
    print("\n⚠️ No API keys found!")
    print("Please set at least one API key in your .env file:")
    print("  GOOGLE_API_KEY=your_key_here")
    print("  OPENAI_API_KEY=your_key_here") 
    print("  ANTHROPIC_API_KEY=your_key_here")
else:
    print(f"\n✅ {sum(api_status.values())} out of {len(api_status)} models available!")

🔑 API Keys Status Check
Available models:
  gemini: ✅ Available
  openai: ❌ Missing API Key
  anthropic: ❌ Missing API Key

Recommended model: gemini

✅ 1 out of 3 models available!


### Testing Image Format Support

Verify that JPG and other image formats are properly supported and test MIME type handling for different file extensions.

In [ ]:
# Test JPG file support and MIME type handling
print("🔍 Testing JPG File Support")
print("=" * 40)

# Show supported formats
print("Supported image formats:")
for fmt in sorted(ocr.supported_formats):
    print(f"  {fmt}")

print("\nMIME type mappings:")
for ext, mime in ocr.mime_type_map.items():
    print(f"  {ext} → image/{mime}")

print("\n📁 Checking for JPG files in your directories...")

# Check both directories for JPG files
directories_to_check = [
    "/home/tanjim_noor/Work/AI OCR BenchMark/Pay Slip",
    "/home/tanjim_noor/Work/AI OCR BenchMark/test png"
]

for directory in directories_to_check:
    if os.path.exists(directory):
        print(f"\n📂 {directory}:")
        try:
            all_files = list(Path(directory).iterdir())
            jpg_files = [f for f in all_files if f.suffix.lower() in ['.jpg', '.jpeg']]
            
            if jpg_files:
                print(f"  Found {len(jpg_files)} JPG/JPEG file(s):")
                for jpg_file in jpg_files[:5]:  # Show first 5
                    file_ext = jpg_file.suffix.lower()
                    mime_type = ocr.mime_type_map.get(file_ext, file_ext[1:])
                    print(f"    {jpg_file.name} ({file_ext} → image/{mime_type})")
                if len(jpg_files) > 5:
                    print(f"    ... and {len(jpg_files) - 5} more")
            else:
                print("  No JPG files found")
                
            # Show all supported image files
            supported_images = ocr.get_images_from_directory(directory)
            if supported_images:
                print(f"  Total supported images: {len(supported_images)}")
        except Exception as e:
            print(f"  Error reading directory: {e}")
    else:
        print(f"\n📂 {directory}: Directory not found")

print(f"\n✅ JPG files are fully supported!")
print("The system will automatically convert .jpg extensions to 'image/jpeg' MIME type.")

### MIME Type Mapping Test

In [ ]:
# Quick MIME type test
print("🧪 Testing MIME Type Mapping")
print("=" * 30)

test_extensions = ['.jpg', '.jpeg', '.png', '.webp', '.gif']
for ext in test_extensions:
    mime_type = ocr.mime_type_map.get(ext, ext[1:])
    print(f"{ext} → image/{mime_type}")

# Test with your specific file
test_file = "/home/tanjim_noor/Work/AI OCR BenchMark/Pay Slip/IMG_20250826_154052.jpg"
if os.path.exists(test_file):
    file_ext = Path(test_file).suffix.lower()
    mime_type = ocr.mime_type_map.get(file_ext, file_ext[1:])
    print(f"\n📁 Your file: {Path(test_file).name}")
    print(f"Extension: {file_ext}")
    print(f"MIME type: image/{mime_type}")
    print(f"Should be: image/jpeg")
    print(f"Mapping works: {'✅' if mime_type == 'jpeg' else '❌'}")
else:
    print(f"\n📁 Test file not found: {test_file}")

🧪 Testing MIME Type Mapping
.jpg → image/jpeg
.jpeg → image/jpeg
.png → image/png
.webp → image/webp
.gif → image/gif

📁 Your file: IMG_20250826_154052.jpg
Extension: .jpg
MIME type: image/jpeg
Should be: image/jpeg
Mapping works: ✅


### Image Diagnostics

Analyze image files for size, dimensions, and optimization requirements to understand processing needs.

In [ ]:
# Diagnostic: Check image file sizes and dimensions
print("🔍 Image Diagnostics")
print("=" * 40)

pay_slip_dir = "/home/tanjim_noor/Work/AI OCR BenchMark/Pay Slip"
if os.path.exists(pay_slip_dir):
    image_files = ocr.get_images_from_directory(pay_slip_dir)
    
    if image_files:
        print(f"Found {len(image_files)} images. Analyzing first few...")
        
        for i, img_path in enumerate(image_files[:3]):
            print(f"\n📁 {i+1}. {Path(img_path).name}")
            
            # File size
            file_size_mb = os.path.getsize(img_path) / (1024 * 1024)
            print(f"   File size: {file_size_mb:.2f} MB")
            
            # Image dimensions
            try:
                with Image.open(img_path) as img:
                    print(f"   Dimensions: {img.size[0]}x{img.size[1]} pixels")
                    print(f"   Mode: {img.mode}")
                    
                    # Estimate base64 size
                    estimated_base64_mb = file_size_mb * 1.33  # Base64 is ~33% larger
                    print(f"   Estimated base64 size: {estimated_base64_mb:.2f} MB")
                    
                    if file_size_mb > 4:
                        print(f"   ⚠️ Large file - will be optimized")
                    if img.size[0] > 1024 or img.size[1] > 1024:
                        print(f"   ⚠️ Large dimensions - will be resized")
                        
            except Exception as e:
                print(f"   ❌ Error reading image: {e}")
    else:
        print("No images found")
else:
    print("Directory not found")

🔍 Image Diagnostics
Found 25 images. Analyzing first few...

📁 1. IMG_20250826_154052.jpg
   File size: 2.98 MB
   Dimensions: 4624x3472 pixels
   Mode: RGB
   Estimated base64 size: 3.96 MB
   ⚠️ Large dimensions - will be resized

📁 2. IMG_20250826_154131.jpg
   File size: 3.70 MB
   Dimensions: 4624x3472 pixels
   Mode: RGB
   Estimated base64 size: 4.92 MB
   ⚠️ Large dimensions - will be resized

📁 3. IMG_20250826_154154.jpg
   File size: 3.52 MB
   Dimensions: 4624x3472 pixels
   Mode: RGB
   Estimated base64 size: 4.69 MB
   ⚠️ Large dimensions - will be resized


### Image Optimization Configuration

Utilities to configure global image optimization settings including resize ratios, quality settings, and file size limits.

In [ ]:
# Image Optimization Configuration Utility
print("🛠️ Image Optimization Configuration")
print("=" * 50)

def update_global_settings(enable_optimization=None, resize_ratio=None, max_dimension=None, 
                          max_file_size_mb=None, jpeg_quality=None):
    """Update global image optimization settings."""
    global ENABLE_IMAGE_OPTIMIZATION, IMAGE_RESIZE_RATIO, MAX_IMAGE_DIMENSION
    global MAX_FILE_SIZE_MB, JPEG_QUALITY
    
    if enable_optimization is not None:
        ENABLE_IMAGE_OPTIMIZATION = enable_optimization
        print(f"✅ Global optimization: {'Enabled' if enable_optimization else 'Disabled'}")
    
    if resize_ratio is not None:
        if 0.1 <= resize_ratio <= 1.0:
            IMAGE_RESIZE_RATIO = resize_ratio
            print(f"✅ Global resize ratio: {resize_ratio}")
        else:
            print(f"❌ Invalid resize ratio: {resize_ratio} (must be 0.1-1.0)")
    
    if max_dimension is not None:
        MAX_IMAGE_DIMENSION = max_dimension
        print(f"✅ Global max dimension: {max_dimension}px")
    
    if max_file_size_mb is not None:
        MAX_FILE_SIZE_MB = max_file_size_mb
        print(f"✅ Global max file size: {max_file_size_mb}MB")
    
    if jpeg_quality is not None:
        if 1 <= jpeg_quality <= 100:
            JPEG_QUALITY = jpeg_quality
            print(f"✅ Global JPEG quality: {jpeg_quality}")
        else:
            print(f"❌ Invalid JPEG quality: {jpeg_quality} (must be 1-100)")

def show_global_settings():
    """Display current global settings."""
    print("🌐 Current Global Settings:")
    print(f"  ENABLE_IMAGE_OPTIMIZATION = {ENABLE_IMAGE_OPTIMIZATION}")
    print(f"  IMAGE_RESIZE_RATIO = {IMAGE_RESIZE_RATIO}")
    print(f"  MAX_IMAGE_DIMENSION = {MAX_IMAGE_DIMENSION}")
    print(f"  MAX_FILE_SIZE_MB = {MAX_FILE_SIZE_MB}")
    print(f"  JPEG_QUALITY = {JPEG_QUALITY}")

# Show current settings
show_global_settings()

print("\n📖 Usage Examples:")
print("# Disable optimization completely:")
print("update_global_settings(enable_optimization=False)")
print("\n# Resize images to 25% of original size:")
print("update_global_settings(resize_ratio=0.25)")
print("\n# Set maximum dimension to 512px:")
print("update_global_settings(max_dimension=512)")
print("\n# Multiple settings at once:")
print("update_global_settings(enable_optimization=True, resize_ratio=0.3, jpeg_quality=70)")

### Enhanced Processing with Formatting

Execute enhanced batch processing with improved JSON formatting and structured data extraction for better analysis.

In [ ]:
# Enhanced processing with better formatting
try:
    pay_slip_dir = "/home/tanjim_noor/Work/AI OCR BenchMark/For benchmark"
    output_file = "/home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output/ocr_result_formatted.json"
    
    print("🚀 Starting enhanced batch processing with formatting...")
    print(f"Input directory: {pay_slip_dir}")
    print(f"Output file: {output_file}")
    print("=" * 50)
    
    # Process all images with payslip extraction prompt
    results = ocr.process_directory(
        directory_path=pay_slip_dir,
        prompt=sample_prompts["cash_expense_extraction_V2"],
        output_file=output_file,
        save_formatted=True  # Enable enhanced formatting
    )
    
    # Summary
    successful = sum(1 for r in results if r["success"])
    failed = len(results) - successful
    parsed_json = sum(1 for r in results if "extracted_data_parsed" in r)
    
    print("=" * 50)
    print(f"📊 ENHANCED PROCESSING SUMMARY:")
    print(f"Total images: {len(results)}")
    print(f"Successful: {successful}")
    print(f"Failed: {failed}")
    print(f"Successfully parsed JSON: {parsed_json}")
    print(f"📁 Files created:")
    print(f"  • Main results: {output_file}")
    print(f"  • Structured data only: {output_file.replace('.json', '_structured_only.json')}")
    
    # Show sample of structured data
    if results and "extracted_data_parsed" in results[0]:
        print(f"\n📋 Sample structured data from first image:")
        sample_data = results[0]["extracted_data_parsed"]
        print(json.dumps(sample_data, indent=2)[:500] + "..." if len(str(sample_data)) > 500 else json.dumps(sample_data, indent=2))
    
except Exception as e:
    print(f"Error during enhanced batch processing: {e}")

### Utility Function to view formatted results

View the formatted results

In [ ]:
# Utility function to view formatted results
def view_formatted_results(results_file: str, show_full_data: bool = False):
    """
    View formatted OCR results in a readable way.
    
    Args:
        results_file: Path to the JSON results file
        show_full_data: Whether to show complete structured data or just summary
    """
    try:
        with open(results_file, 'r', encoding='utf-8') as f:
            results = json.load(f)
        
        print(f"📊 OCR Results Summary from {Path(results_file).name}")
        print("=" * 60)
        
        for i, result in enumerate(results, 1):
            print(f"\n🖼️ Image {i}: {Path(result['image_path']).name}")
            print(f"   Model: {result.get('model_used', 'Unknown')}")
            print(f"   Status: {'✅ Success' if result.get('success') else '❌ Failed'}")
            
            if result.get('success'):
                if 'structured_data' in result:
                    structured = result['structured_data']
                    print(f"   Format: 📋 Structured JSON")
                    
                    # Show key information from structured data
                    if isinstance(structured, dict):
                        if 'payment_details' in structured:
                            pd = structured['payment_details']
                            print(f"   Supplier: {pd.get('supplier', 'N/A')}")
                            print(f"   Date: {pd.get('payment_date', 'N/A')}")
                            print(f"   Total: {structured.get('totals', {}).get('total_amount', 'N/A')}")
                            print(f"   Items: {len(structured.get('item_details', []))}")
                        
                        if show_full_data:
                            print(f"\n   📋 Full Structured Data:")
                            print(json.dumps(structured, indent=4))
                    
                elif 'extraction_format' in result:
                    print(f"   Format: 📝 {result['extraction_format'].upper()}")
                    if 'extracted_data_preview' in result:
                        print(f"   Preview: {result['extracted_data_preview']}")
                elif 'extracted_data' in result:
                    # Fallback for older format
                    preview = result['extracted_data'][:100] + "..." if len(result['extracted_data']) > 100 else result['extracted_data']
                    print(f"   Data Preview: {preview}")
            else:
                print(f"   Error: {result.get('error', 'Unknown error')}")
        
        print(f"\n📈 Summary: {sum(1 for r in results if r.get('success'))} successful out of {len(results)} total")
        
    except Exception as e:
        print(f"❌ Error reading results file: {e}")

print("🔍 Enhanced result viewing utility created!")
print("\nUsage examples:")
print("# View summary of formatted results:")
print("view_formatted_results('/home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output/ocr_result_formatted.json')")
print("\n# View with full structured data:")
print("view_formatted_results('/home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output/ocr_result_formatted.json', show_full_data=True)")

🔍 Enhanced result viewing utility created!

Usage examples:
# View summary of formatted results:
view_formatted_results('/home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output/ocr_result_formatted.json')

# View with full structured data:
view_formatted_results('/home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output/ocr_result_formatted.json', show_full_data=True)


# Comprehensive OCR Benchmark Function

This section implements a comprehensive all-rounder function that combines batch processing, scoring, and reporting for multiple prompts and models. The function flow:

1. **Model + Prompt + Image Directory** → OCR Processing
2. **OCR Results + Ground Truth** → Batch Scoring  
3. **Scores Analysis** → Generate Comprehensive Report

Features:
- ✅ Multi-prompt evaluation
- ✅ Batch OCR processing
- ✅ Automated scoring against ground truth
- ✅ Comprehensive performance reports
- ✅ Best prompt identification
- ✅ Model performance comparison

In [8]:
def comprehensive_ocr_benchmark(
    model_name: str,
    prompts_dict: Dict[str, str],
    image_directory: str,
    ground_truth_file: str,
    output_base_dir: str = "/home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output",
    scoring_config: Optional[List[Dict]] = None,
    temperature: float = 0.1
) -> Dict[str, Any]:
    """
    Comprehensive OCR benchmark function that processes images with multiple prompts,
    scores results against ground truth, and generates performance reports.
    
    Args:
        model_name: The AI model to use ('gemini', 'openai', 'anthropic')
        prompts_dict: Dictionary of {prompt_name: prompt_text} to test
        image_directory: Path to directory containing images to process
        ground_truth_file: Path to JSON file with ground truth data
        output_base_dir: Base directory for saving results
        scoring_config: Configuration for scoring system (uses default if None)
        temperature: Model temperature setting
        
    Returns:
        Dictionary containing comprehensive benchmark results including:
        - Individual prompt performance
        - Best performing prompt
        - Detailed scoring breakdown
        - Performance statistics
    """
    
    from datetime import datetime
    import os
    from pathlib import Path
    
    print("🚀 COMPREHENSIVE OCR BENCHMARK")
    print("=" * 60)
    print(f"Model: {model_name}")
    print(f"Image Directory: {image_directory}")
    print(f"Ground Truth: {ground_truth_file}")
    print(f"Prompts to test: {len(prompts_dict)}")
    print(f"Output Directory: {output_base_dir}")
    print("=" * 60)
    
    # Ensure output directory exists
    os.makedirs(output_base_dir, exist_ok=True)
    
    # Initialize OCR system
    try:
        ocr_system = OCRBenchmark(model_name=model_name, temperature=temperature)
        print(f"✅ OCR system initialized with {model_name}")
    except Exception as e:
        print(f"❌ Failed to initialize OCR system: {e}")
        return {"error": f"Failed to initialize OCR system: {e}"}
    
    # Check if ground truth file exists
    if not os.path.exists(ground_truth_file):
        print(f"❌ Ground truth file not found: {ground_truth_file}")
        return {"error": f"Ground truth file not found: {ground_truth_file}"}
    
    # Get list of images to process
    try:
        image_files = ocr_system.get_images_from_directory(image_directory)
        print(f"📂 Found {len(image_files)} images to process")
        if not image_files:
            return {"error": "No supported image files found in directory"}
    except Exception as e:
        print(f"❌ Error reading image directory: {e}")
        return {"error": f"Error reading image directory: {e}"}
    
    # Use default scoring config if none provided
    if scoring_config is None:
        scoring_config = [
            {"path": "payment_details.supplier", "weight": 1, "comparator": "exact"},
            {"path": "payment_details.payment_date", "weight": 1, "comparator": "exact"},
            {"path": "payment_details.payment_account", "weight": 1, "comparator": "embedding"},
            {"path": "payment_details.payment_method", "weight": 1, "comparator": "exact"},
            {"path": "payment_details.ref_no", "weight": 1, "comparator": "exact"},
            {"path": "payment_details.tags", "weight": 1, "comparator": "semantic"},
            {"path": "item_details.category", "weight": 1, "comparator": "exact"},
            {"path": "item_details.description", "weight": 1, "comparator": "embedding"},
            {"path": "item_details.quantity", "weight": 1, "comparator": "numeric", "tolerance": 0.05},
            {"path": "item_details.unit_price", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
            {"path": "item_details.total", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
            {"path": "attachment", "weight": 1, "comparator": "exact"},
            {"path": "memo", "weight": 1, "comparator": "embedding"},
            {"path": "totals.sub_total", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
            {"path": "totals.sales_tax.is_percentage", "weight": 1, "comparator": "exact"},
            {"path": "totals.sales_tax.tax", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
            {"path": "totals.sales_tax.amount", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
            {"path": "totals.discount.is_percentage", "weight": 1, "comparator": "exact"},
            {"path": "totals.discount.discount", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
            {"path": "totals.discount.calculated_amount", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
            {"path": "totals.total_amount", "weight": 1, "comparator": "numeric", "tolerance": 0.1},
        ]
    
    # Store results for all prompts
    prompt_results = {}
    prompt_scores = {}
    
    # Process each prompt
    for prompt_name, prompt_text in prompts_dict.items():
        print(f"\n🔄 Processing prompt: {prompt_name}")
        print("-" * 40)
        
        # Create output file path for this prompt
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = os.path.join(output_base_dir, f"ocr_results_{prompt_name}_{model_name}_{timestamp}.json")
        
        try:
            # Step 1: Batch OCR Processing
            print(f"📝 Step 1: OCR Processing with {model_name}")
            ocr_results = ocr_system.process_directory(
                directory_path=image_directory,
                prompt=prompt_text,
                output_file=output_file,
                save_formatted=True
            )
            
            # Check if processing was successful
            successful_ocr = sum(1 for r in ocr_results if r.get("success", False))
            print(f"✅ OCR completed: {successful_ocr}/{len(ocr_results)} images processed successfully")
            
            if successful_ocr == 0:
                print(f"⚠️ No images processed successfully for prompt: {prompt_name}")
                prompt_results[prompt_name] = {
                    "error": "No images processed successfully",
                    "ocr_results": ocr_results
                }
                continue
            
            # Step 2: Batch Scoring
            print(f"📊 Step 2: Scoring against ground truth")
            structured_results_file = output_file.replace('.json', '_structured_only.json')
            
            if os.path.exists(structured_results_file):
                scoring_results = score_all_images(
                    results_json_path=structured_results_file,
                    actual_json_path=ground_truth_file,
                    config=scoring_config
                )
                
                if scoring_results:
                    avg_score = scoring_results['average_score']
                    print(f"✅ Scoring completed: Average score = {avg_score:.3f}")
                    
                    # Store results
                    prompt_results[prompt_name] = {
                        "ocr_results": ocr_results,
                        "scoring_results": scoring_results,
                        "output_files": {
                            "ocr_results": output_file,
                            "structured_only": structured_results_file
                        },
                        "statistics": {
                            "total_images": len(ocr_results),
                            "successful_ocr": successful_ocr,
                            "processed_for_scoring": scoring_results['processed_count'],
                            "average_score": avg_score
                        }
                    }
                    prompt_scores[prompt_name] = avg_score
                else:
                    print(f"❌ Scoring failed for prompt: {prompt_name}")
                    prompt_results[prompt_name] = {
                        "error": "Scoring failed",
                        "ocr_results": ocr_results
                    }
            else:
                print(f"❌ Structured results file not found: {structured_results_file}")
                prompt_results[prompt_name] = {
                    "error": "Structured results file not found",
                    "ocr_results": ocr_results
                }
                
        except Exception as e:
            print(f"❌ Error processing prompt {prompt_name}: {e}")
            prompt_results[prompt_name] = {
                "error": str(e),
                "ocr_results": []
            }
    
    # Step 3: Generate Comprehensive Report
    print(f"\n📈 Step 3: Generating Comprehensive Report")
    print("=" * 60)
    
    # Initialize variables to avoid UnboundLocalError
    sorted_prompts = []
    best_prompt = None
    
    # Find best performing prompt
    if prompt_scores:
        best_prompt = max(prompt_scores.items(), key=lambda x: x[1])
        worst_prompt = min(prompt_scores.items(), key=lambda x: x[1])
        
        print(f"🏆 PERFORMANCE RANKING:")
        sorted_prompts = sorted(prompt_scores.items(), key=lambda x: x[1], reverse=True)
        for i, (prompt_name, score) in enumerate(sorted_prompts, 1):
            print(f"  {i}. {prompt_name}: {score:.3f}")
        
        print(f"\n🥇 Best Prompt: {best_prompt[0]} (Score: {best_prompt[1]:.3f})")
        print(f"🥉 Worst Prompt: {worst_prompt[0]} (Score: {worst_prompt[1]:.3f})")
        print(f"📊 Score Range: {worst_prompt[1]:.3f} - {best_prompt[1]:.3f}")
        print(f"📈 Score Improvement: {((best_prompt[1] - worst_prompt[1]) / worst_prompt[1] * 100):.1f}%")
    else:
        print("❌ No successful prompt processing to rank")
    
    # Create comprehensive results
    comprehensive_results = {
        "benchmark_metadata": {
            "model_name": model_name,
            "image_directory": image_directory,
            "ground_truth_file": ground_truth_file,
            "timestamp": datetime.now().isoformat(),
            "total_prompts_tested": len(prompts_dict),
            "successful_prompts": len(prompt_scores),
            "total_images": len(image_files)
        },
        "prompt_results": prompt_results,
        "performance_ranking": sorted_prompts if prompt_scores else [],
        "best_prompt": {
            "name": best_prompt[0] if best_prompt else None,
            "score": best_prompt[1] if best_prompt else None
        } if best_prompt else None,
        "summary_statistics": {
            "prompt_scores": prompt_scores,
            "average_score_across_prompts": sum(prompt_scores.values()) / len(prompt_scores) if prompt_scores else 0,
            "score_std_deviation": float(np.std(list(prompt_scores.values()))) if prompt_scores else 0
        }
    }
    
    # Save comprehensive results to file
    comprehensive_output_file = os.path.join(output_base_dir, f"comprehensive_benchmark_{model_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")
    with open(comprehensive_output_file, 'w') as f:
        json.dump(comprehensive_results, f, indent=2)
    
    print(f"💾 Comprehensive results saved to: {comprehensive_output_file}")
    print("=" * 60)
    print("🎉 BENCHMARK COMPLETED!")
    
    return comprehensive_results

In [ ]:
# Helper function to create test prompt sets
def create_test_prompt_sets():
    """Create different sets of prompts for testing various scenarios."""
    
    # Core prompts from the existing sample_prompts
    core_prompts = {
        "cash_expense_v2": sample_prompts["cash_expense_extraction_V2"],
        "cash_expense_v3": sample_prompts["cash_expense_extraction_V3"],
    }
    
    # Experimental prompts for testing variations
    experimental_prompts = {
        "detailed_extraction": """
You are an expert document processing assistant. Extract ALL financial data from this invoice/receipt image.
Focus on accuracy and completeness. Return data in structured JSON format with these sections:
- payment_details (supplier, date, method, reference)
- item_details (all line items with quantities, prices, totals)
- totals (subtotal, tax, discount, final total)
- additional_info (any other relevant data)

Be extremely precise with numbers and calculations. If text is in Bengali/Bangla, transliterate to English.
Return ONLY valid JSON, no additional text.
        """,
        
        "simplified_extraction": """
Extract key information from this receipt/invoice:
1. Vendor/Store name
2. Date
3. Total amount
4. Payment method
5. List of items purchased

Format as clean JSON. Focus on the most important information.
        """,
        
        "validation_focused": """
Extract data from this financial document. Pay special attention to:
- Mathematical accuracy (ensure totals match item sums)
- Proper data types (numbers as numbers, dates as dates)
- Complete item extraction (don't miss any line items)
- Tax and discount calculations

Return as structured JSON following this exact schema:
{payment_details: {}, item_details: [], totals: {}}
        """
    }
    
    return {
        "core_set": core_prompts,
        "experimental_set": experimental_prompts,
        "combined_set": {**core_prompts, **experimental_prompts}
    }

# Helper function to run quick benchmark
def quick_benchmark(model_name: str = "gemini", prompt_set: str = "core_set"):
    """
    Run a quick benchmark with predefined settings.
    
    Args:
        model_name: Model to use ('gemini', 'openai', 'anthropic')
        prompt_set: Which prompt set to use ('core_set', 'experimental_set', 'combined_set')
    """
    
    # Get prompts
    prompt_sets = create_test_prompt_sets()
    prompts_to_test = prompt_sets.get(prompt_set, prompt_sets["core_set"])
    
    # Default paths
    image_directory = "/home/tanjim_noor/Work/AI OCR BenchMark/For benchmark"
    ground_truth_file = "/home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output/ocr_actual_1_structured_only.json"
    
    print(f"🚀 Quick Benchmark: {model_name} with {prompt_set}")
    print(f"Testing {len(prompts_to_test)} prompts on images from: {image_directory}")
    
    # Run comprehensive benchmark
    results = comprehensive_ocr_benchmark(
        model_name=model_name,
        prompts_dict=prompts_to_test,
        image_directory=image_directory,
        ground_truth_file=ground_truth_file
    )
    
    return results

# Helper function to display benchmark results nicely
def display_benchmark_summary(results: Dict[str, Any]):
    """Display a nice summary of benchmark results."""
    
    if "error" in results:
        print(f"❌ Benchmark failed: {results['error']}")
        return
    
    metadata = results["benchmark_metadata"]
    
    print("\n" + "=" * 80)
    print("🏆 COMPREHENSIVE BENCHMARK SUMMARY")
    print("=" * 80)
    
    print(f"📊 Model: {metadata['model_name']}")
    print(f"📂 Images: {metadata['total_images']} total")
    print(f"📝 Prompts: {metadata['total_prompts_tested']} tested, {metadata['successful_prompts']} successful")
    print(f"⏰ Timestamp: {metadata['timestamp']}")
    
    if results["best_prompt"]:
        print(f"\n🥇 Best Performing Prompt:")
        print(f"   Name: {results['best_prompt']['name']}")
        print(f"   Score: {results['best_prompt']['score']:.3f}")
    
    print(f"\n📈 Performance Ranking:")
    for i, (prompt_name, score) in enumerate(results["performance_ranking"], 1):
        print(f"   {i}. {prompt_name}: {score:.3f}")
    
    stats = results["summary_statistics"]
    print(f"\n📊 Summary Statistics:")
    print(f"   Average Score: {stats['average_score_across_prompts']:.3f}")
    print(f"   Score Std Dev: {stats['score_std_deviation']:.3f}")
    
    print(f"\n💾 Results Location: Check OCR_Output directory for detailed files")

print("🛠️ Helper functions created!")
print("\nUsage Examples:")
print("# Quick test with core prompts and Gemini:")
print("results = quick_benchmark('gemini', 'core_set')")
print("display_benchmark_summary(results)")
print("\n# Full custom benchmark:")
print("my_prompts = {'test1': 'Extract all text', 'test2': 'Extract prices only'}")
print("results = comprehensive_ocr_benchmark('gemini', my_prompts, '/path/to/images', '/path/to/ground_truth.json')")

🛠️ Helper functions created!

Usage Examples:
# Quick test with core prompts and Gemini:
results = quick_benchmark('gemini', 'core_set')
display_benchmark_summary(results)

# Full custom benchmark:
my_prompts = {'test1': 'Extract all text', 'test2': 'Extract prices only'}
results = comprehensive_ocr_benchmark('gemini', my_prompts, '/path/to/images', '/path/to/ground_truth.json')


## 🎯 Demo: Comprehensive Benchmark Usage

This section demonstrates how to use the comprehensive benchmark function to test multiple prompts and identify the best performing one. The demo will:

1. **Load multiple prompts** from predefined sets
2. **Process all images** in the benchmark directory
3. **Score results** against ground truth data
4. **Generate a report** showing which prompt performs best

Uncomment the code below to run a comprehensive benchmark test.

In [10]:
# Demo: Comprehensive OCR Benchmark
# Uncomment the code below to run a full benchmark test


# Example 1: Quick benchmark with core prompts
print("🚀 Running Quick Benchmark with Core Prompts...")
results = quick_benchmark(model_name="huggingface", prompt_set="core_set")
display_benchmark_summary(results)


"""
# Example 2: Custom comprehensive benchmark
print("🔧 Running Custom Comprehensive Benchmark...")

# Define custom prompts to test
test_prompts = {
    "v1_basic": sample_prompts["cash_expense_extraction"],
    "v2_enhanced": sample_prompts["cash_expense_extraction_V2"],
    "general": sample_prompts["general_ocr"],
    "structured": sample_prompts["structured_data"]
}

# Set paths
image_dir = "/home/tanjim_noor/Work/AI OCR BenchMark/For benchmark"
ground_truth = "/home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output/ocr_actual_1_structured_only.json"

# Run comprehensive benchmark
results = comprehensive_ocr_benchmark(
    model_name="gemini",
    prompts_dict=test_prompts,
    image_directory=image_dir,
    ground_truth_file=ground_truth,
    temperature=0.1
)

# Display results
display_benchmark_summary(results)

# Optionally, access detailed results
if "prompt_results" in results:
    print("\n📊 Detailed Results:")
    for prompt_name, prompt_data in results["prompt_results"].items():
        if "statistics" in prompt_data:
            stats = prompt_data["statistics"]
            print(f"  {prompt_name}:")
            print(f"    OCR Success: {stats['successful_ocr']}/{stats['total_images']}")
            print(f"    Scored Images: {stats['processed_for_scoring']}")
            print(f"    Average Score: {stats['average_score']:.3f}")
"""

"""
# Example 3: Compare multiple models on same prompts
print("🔬 Multi-Model Comparison...")

models_to_test = ["gemini"]  # Add "openai", "anthropic" if you have API keys
test_prompts = {
    "cash_v2": sample_prompts["cash_expense_extraction_V2"],
    "general": sample_prompts["general_ocr"]
}

model_comparison = {}

for model in models_to_test:
    print(f"\n🤖 Testing {model}...")
    try:
        model_results = comprehensive_ocr_benchmark(
            model_name=model,
            prompts_dict=test_prompts,
            image_directory="/home/tanjim_noor/Work/AI OCR BenchMark/For benchmark",
            ground_truth_file="/home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output/ocr_actual_1_structured_only.json"
        )
        model_comparison[model] = model_results
        
        if model_results.get("best_prompt"):
            best = model_results["best_prompt"]
            print(f"   Best prompt for {model}: {best['name']} (Score: {best['score']:.3f})")
            
    except Exception as e:
        print(f"   ❌ {model} failed: {e}")
        model_comparison[model] = {"error": str(e)}

# Summary across models
print("\n🏆 MULTI-MODEL SUMMARY:")
for model, results in model_comparison.items():
    if "error" not in results and results.get("best_prompt"):
        best = results["best_prompt"]
        avg_score = results["summary_statistics"]["average_score_across_prompts"]
        print(f"  {model}: Best = {best['name']} ({best['score']:.3f}), Avg = {avg_score:.3f}")
    else:
        print(f"  {model}: Failed or no results")
"""

print("🔍 Demo code ready!")
print("Uncomment the sections above to run different types of benchmarks:")
print("  - Example 1: Quick test with predefined prompts")
print("  - Example 2: Custom comprehensive benchmark") 
print("  - Example 3: Multi-model comparison")
print("\n⚠️ Note: Each benchmark processes all images and makes API calls.")
print("Make sure you have sufficient API credits before running extensive tests.")

🚀 Running Quick Benchmark with Core Prompts...
🚀 Quick Benchmark: huggingface with core_set
Testing 4 prompts on images from: /home/tanjim_noor/Work/AI OCR BenchMark/For benchmark
🚀 COMPREHENSIVE OCR BENCHMARK
Model: huggingface
Image Directory: /home/tanjim_noor/Work/AI OCR BenchMark/For benchmark
Ground Truth: /home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output/ocr_actual_1_structured_only.json
Prompts to test: 4
Output Directory: /home/tanjim_noor/Work/AI OCR BenchMark/OCR_Output
✅ OCR system initialized with huggingface
📂 Found 10 images to process

🔄 Processing prompt: cash_expense_v1
----------------------------------------
📝 Step 1: OCR Processing with huggingface
Found 10 image(s) to process...
Processing image 1/10: IMG_20250826_154308~2.jpg
🚀 Starting image processing...
🔍 File extension: .jpg → MIME type: image/jpeg
🖼️ Optimizing image...
📏 Original file size: 1.33 MB
📐 Original dimensions: 3472x4624
🔧 HuggingFace aggressive resize by ratio 0.5: 1736x2312
💾 HuggingFace JPEG q

#  Implementation Summary & Quick Reference

## ✅ **What We've Implemented**

###  **Comprehensive OCR Benchmark Function**
A complete all-rounder function that implements your requested workflow:

**Model + Prompts + Image Directory → OCR Processing → Scoring → Report Generation**

### 🔧 **Key Features Delivered:**

1. **`comprehensive_ocr_benchmark()`** - Main function that:
   - ✅ Takes model, prompts dict, and image directory as input
   - ✅ Processes all images in batches with each prompt
   - ✅ Outputs results to JSON files (structured format)
   - ✅ Automatically scores against ground truth data
   - ✅ Generates comprehensive performance reports
   - ✅ Identifies best performing prompt

2. **Helper Functions:**
   - ✅ `quick_benchmark()` - Easy testing with predefined prompts
   - ✅ `create_test_prompt_sets()` - Predefined prompt collections
   - ✅ `display_benchmark_summary()` - Pretty result display

3. **Multi-Prompt Testing:**
   - ✅ Core prompts (existing cash_expense_extraction variants)
   - ✅ Experimental prompts (detailed, simplified, validation-focused)
   - ✅ Custom prompt support

## **Quick Usage Guide**

### **Simple Test:**
```python
# Quick test with predefined prompts
results = quick_benchmark("gemini", "core_set")
display_benchmark_summary(results)
```

### **Custom Comprehensive Test:**
```python
# Define your prompts
my_prompts = {
    "prompt1": "Extract all invoice data...",
    "prompt2": "Focus on total amounts...",
    "prompt3": "Extract supplier info..."
}

# Run comprehensive benchmark
results = comprehensive_ocr_benchmark(
    model_name="gemini",
    prompts_dict=my_prompts,
    image_directory="/path/to/images",
    ground_truth_file="/path/to/ground_truth.json"
)

# View results
display_benchmark_summary(results)
```

### **Multi-Model Comparison:**
```python
models = ["gemini", "openai", "anthropic"]
for model in models:
    results = comprehensive_ocr_benchmark(model, my_prompts, ...)
    print(f"{model} best: {results['best_prompt']['name']}")
```

## 📊 **Output Files Generated:**
- `ocr_results_{prompt}_{model}_{timestamp}.json` - Full OCR results
- `ocr_results_{prompt}_{model}_{timestamp}_structured_only.json` - Structured data only
- `comprehensive_benchmark_{model}_{timestamp}.json` - Complete benchmark report

## 🏆 **Report Contents:**
- **Performance Ranking** - Prompts sorted by score
- **Best Prompt Identification** - Highest scoring prompt
- **Statistical Analysis** - Average scores, standard deviation
- **Detailed Breakdown** - Per-image, per-field scoring
- **Success Rates** - OCR processing and scoring success rates

## 🔄 **Complete Workflow Implemented:**
1. **Input:** Model + Prompt Set + Image Directory
2. **Processing:** Batch OCR for each prompt
3. **Scoring:** Compare with ground truth using multiple metrics
4. **Analysis:** Statistical analysis and ranking
5. **Output:** Comprehensive report identifying best prompt

**✅ Your requested flow has been fully implemented and is ready to use!**